<a href="https://colab.research.google.com/github/SarahVSargent/stock_signals_regressions/blob/main/Fama_French_Five_Factor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mods to three factor

In [ ]:
#rmw- robust minus weak (profitability)
##operative provitiabulity= (revenue- expenses)/ book value

#cma conservative aggressive (investment)
##investment= assets (reference year-1)/ assets (reference year-2)

Intro

In [ ]:
##
#coonnect google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Import tools
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

!pip install pandasql

import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats.mstats import winsorize
import math
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import pandasql as ps

In [ ]:
file = "/content/drive/MyDrive/Exeter google drive /internships jobs/FIMA '23/Fama-french 5 factor/CRSPMonthly1990Through2022.csv"
x = pd.read_csv(file)
print(x.columns)

file = "/content/drive/MyDrive/Exeter google drive /internships jobs/FIMA '23/Fama-french 5 factor/ccmfund1990to2022.csv"
y = pd.read_csv(file)
print(y)

print("imported")

<ipython-input-4-b899753278ca>:2: DtypeWarning: Columns (5,9,18,19,49,58) have mixed types. Specify dtype option on import or set low_memory=False.
  x = pd.read_csv(file)


Index(['PERMNO', 'date', 'NAMEENDT', 'SHRCD', 'EXCHCD', 'SICCD', 'NCUSIP',
       'TICKER', 'COMNAM', 'SHRCLS', 'TSYMBOL', 'NAICS', 'PRIMEXCH', 'TRDSTAT',
       'SECSTAT', 'PERMCO', 'ISSUNO', 'HEXCD', 'HSICCD', 'CUSIP', 'DCLRDT',
       'DLAMT', 'DLPDT', 'DLSTCD', 'NEXTDT', 'PAYDT', 'RCRDDT', 'SHRFLG',
       'HSICMG', 'HSICIG', 'DISTCD', 'DIVAMT', 'FACPR', 'FACSHR', 'ACPERM',
       'ACCOMP', 'SHRENDDT', 'NWPERM', 'DLRETX', 'DLPRC', 'DLRET', 'TRTSCD',
       'NMSIND', 'MMCNT', 'NSDINX', 'BIDLO', 'ASKHI', 'PRC', 'VOL', 'RET',
       'BID', 'ASK', 'SHROUT', 'CFACPR', 'CFACSHR', 'ALTPRC', 'SPREAD',
       'ALTPRCDT', 'RETX', 'vwretd', 'vwretx', 'ewretd', 'ewretx', 'sprtrn'],
      dtype='object')


<ipython-input-4-b899753278ca>:6: DtypeWarning: Columns (2,17,19,24,33,37,952,953,955,956,957,962,976,977,982) have mixed types. Specify dtype option on import or set low_memory=False.
  y = pd.read_csv(file)


         GVKEY LINKPRIM LIID LINKTYPE  LPERMNO  LPERMCO    LINKDT LINKENDDT  \
0         1004        P    1       LU    54594    20000  19720424         E   
1         1004        P    1       LU    54594    20000  19720424         E   
2         1004        P    1       LU    54594    20000  19720424         E   
3         1004        P    1       LU    54594    20000  19720424         E   
4         1004        P    1       LU    54594    20000  19720424         E   
...        ...      ...  ...      ...      ...      ...       ...       ...   
209121  345980        P    1       LC    20333    57666  20201216         E   
209122  345980        P    1       LC    20333    57666  20201216         E   
209123  347007        P    1       LC    15533    55364  20210310         E   
209124  347085        P    1       LC    21069    58208  20210401         E   
209125  349530        P    1       LC    17324    56274  20210709         E   

        datadate   fyear  ... priusa     sic spcind

In [ ]:
CRSPMonthly1990Through2022 = x

In [ ]:
ccmfund1990to2022 = y

In [ ]:
print(ccmfund1990to2022)

         GVKEY LINKPRIM LIID LINKTYPE  LPERMNO  LPERMCO    LINKDT LINKENDDT  \
0         1004        P    1       LU    54594    20000  19720424         E   
1         1004        P    1       LU    54594    20000  19720424         E   
2         1004        P    1       LU    54594    20000  19720424         E   
3         1004        P    1       LU    54594    20000  19720424         E   
4         1004        P    1       LU    54594    20000  19720424         E   
...        ...      ...  ...      ...      ...      ...       ...       ...   
209121  345980        P    1       LC    20333    57666  20201216         E   
209122  345980        P    1       LC    20333    57666  20201216         E   
209123  347007        P    1       LC    15533    55364  20210310         E   
209124  347085        P    1       LC    21069    58208  20210401         E   
209125  349530        P    1       LC    17324    56274  20210709         E   

        datadate   fyear  ... priusa     sic spcind

In [ ]:
CRSPMonthly1990Through2022.shape

(2975241, 64)

In [ ]:

file = "/content/drive/MyDrive/Exeter google drive /internships jobs/FIMA '23/Fama-french 5 factor/FamaFrenchFiveFactorsMonthly - FamaFrenchFiveFactorsMonthly.csv"
FamaFrenchFiveFactorsMonthly = pd.read_csv(file)
print(FamaFrenchFiveFactorsMonthly.columns)

Index(['Unnamed: 0', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF'], dtype='object')


clean just CRSP

In [ ]:
##
#4:53

#only keeping these relevant variables
CRSPMonthly1990Through2022= CRSPMonthly1990Through2022[["PERMNO", "date", "RET", "SHROUT", "ALTPRC", "EXCHCD", "SHRCD", "SICCD", "DLRET", "DLSTCD"]]
#"revt","cogs","xint","xsga"

#drop duplicates (instead of slect distinct)
CRSPMonthly1990Through2022.drop_duplicates(inplace=True)

<ipython-input-10-694d5bc39d7b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CRSPMonthly1990Through2022.drop_duplicates(inplace=True)


In [ ]:

print(CRSPMonthly1990Through2022)

         PERMNO      date        RET     SHROUT      ALTPRC  EXCHCD  SHRCD  \
0         10001  19900131  -0.018519     1022.0    -9.93750     3.0   11.0   
1         10001  19900228  -0.006289     1022.0    -9.87500     3.0   11.0   
2         10001  19900330   0.012658     1027.0    -9.87500     3.0   11.0   
3         10001  19900430   0.000000     1027.0    -9.87500     3.0   11.0   
4         10001  19900531  -0.012658     1027.0     9.75000     3.0   11.0   
...         ...       ...        ...        ...         ...     ...    ...   
2975236   93436  20211130   0.027612  1004265.0  1144.76001     3.0   11.0   
2975237   93436  20211231  -0.076855  1033534.0  1056.78003     3.0   11.0   
2975238   93436  20220131  -0.113609  1033534.0   936.71997     3.0   11.0   
2975239   93436  20220228  -0.070768  1033534.0   870.42999     3.0   11.0   
2975240   93436  20220331   0.238009  1033534.0  1077.59998     3.0   11.0   

          SICCD DLRET  DLSTCD  
0          4920   NaN     NaN  

In [ ]:
##
#5:10

#convert variables to numeric- nessesary? Yes, good practice
CRSPMonthly1990Through2022['PERMNO']= pd.to_numeric(CRSPMonthly1990Through2022['PERMNO'], errors= 'coerce')
CRSPMonthly1990Through2022['date']= pd.to_datetime(CRSPMonthly1990Through2022['date'], format='%Y%m%d') # month identifier
CRSPMonthly1990Through2022['RET']= pd.to_numeric(CRSPMonthly1990Through2022['RET'], errors= 'coerce')
CRSPMonthly1990Through2022['SHROUT']= pd.to_numeric(CRSPMonthly1990Through2022['SHROUT'], errors= 'coerce')
CRSPMonthly1990Through2022['ALTPRC']= pd.to_numeric(CRSPMonthly1990Through2022['ALTPRC'], errors= 'coerce')
CRSPMonthly1990Through2022['EXCHCD']= pd.to_numeric(CRSPMonthly1990Through2022['EXCHCD'], errors= 'coerce')
CRSPMonthly1990Through2022['SHRCD']= pd.to_numeric(CRSPMonthly1990Through2022['SHRCD'], errors= 'coerce')
CRSPMonthly1990Through2022['SICCD']= pd.to_numeric(CRSPMonthly1990Through2022['SICCD'], errors= 'coerce')
CRSPMonthly1990Through2022['DLRET']= pd.to_numeric(CRSPMonthly1990Through2022['DLRET'], errors= 'coerce')
CRSPMonthly1990Through2022['DLSTCD']= pd.to_numeric(CRSPMonthly1990Through2022['DLSTCD'], errors= 'coerce')
#convert returns to percent
CRSPMonthly1990Through2022['RET'] *=100
CRSPMonthly1990Through2022['DLRET'] *=100


print(CRSPMonthly1990Through2022)


<ipython-input-12-ff2bb8ab9732>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CRSPMonthly1990Through2022['PERMNO']= pd.to_numeric(CRSPMonthly1990Through2022['PERMNO'], errors= 'coerce')
<ipython-input-12-ff2bb8ab9732>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CRSPMonthly1990Through2022['date']= pd.to_datetime(CRSPMonthly1990Through2022['date'], format='%Y%m%d') # month identifier
<ipython-input-12-ff2bb8ab9732>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

         PERMNO       date      RET     SHROUT      ALTPRC  EXCHCD  SHRCD  \
0         10001 1990-01-31  -1.8519     1022.0    -9.93750     3.0   11.0   
1         10001 1990-02-28  -0.6289     1022.0    -9.87500     3.0   11.0   
2         10001 1990-03-30   1.2658     1027.0    -9.87500     3.0   11.0   
3         10001 1990-04-30   0.0000     1027.0    -9.87500     3.0   11.0   
4         10001 1990-05-31  -1.2658     1027.0     9.75000     3.0   11.0   
...         ...        ...      ...        ...         ...     ...    ...   
2975236   93436 2021-11-30   2.7612  1004265.0  1144.76001     3.0   11.0   
2975237   93436 2021-12-31  -7.6855  1033534.0  1056.78003     3.0   11.0   
2975238   93436 2022-01-31 -11.3609  1033534.0   936.71997     3.0   11.0   
2975239   93436 2022-02-28  -7.0768  1033534.0   870.42999     3.0   11.0   
2975240   93436 2022-03-31  23.8009  1033534.0  1077.59998     3.0   11.0   

          SICCD  DLRET  DLSTCD  
0        4920.0    NaN     NaN  
1        

<ipython-input-12-ff2bb8ab9732>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CRSPMonthly1990Through2022['DLRET']= pd.to_numeric(CRSPMonthly1990Through2022['DLRET'], errors= 'coerce')
<ipython-input-12-ff2bb8ab9732>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CRSPMonthly1990Through2022['DLSTCD']= pd.to_numeric(CRSPMonthly1990Through2022['DLSTCD'], errors= 'coerce')
<ipython-input-12-ff2bb8ab9732>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [ ]:
##
#5:30

#only data rows after 2018

CRSP2018to2022= CRSPMonthly1990Through2022[CRSPMonthly1990Through2022['date']>= "2018-01-01"]

#only data rows in US
#what numbers mean?- only index
CRSP2018to2022= CRSP2018to2022[(CRSP2018to2022['SHRCD']==10) | (CRSP2018to2022['SHRCD']==11)]


print(CRSP2018to2022)

         PERMNO       date      RET     SHROUT      ALTPRC  EXCHCD  SHRCD  \
2363      10026 2018-01-31  -8.8191    18678.0   138.44000     3.0   11.0   
2364      10026 2018-02-28  -2.9688    18678.0   134.33000     3.0   11.0   
2365      10026 2018-03-29   1.9951    18697.0   136.56000     3.0   11.0   
2366      10026 2018-04-30   0.6224    18702.0   137.41000     3.0   11.0   
2367      10026 2018-05-31   3.0638    18702.0   141.62000     3.0   11.0   
...         ...        ...      ...        ...         ...     ...    ...   
2975236   93436 2021-11-30   2.7612  1004265.0  1144.76001     3.0   11.0   
2975237   93436 2021-12-31  -7.6855  1033534.0  1056.78003     3.0   11.0   
2975238   93436 2022-01-31 -11.3609  1033534.0   936.71997     3.0   11.0   
2975239   93436 2022-02-28  -7.0768  1033534.0   870.42999     3.0   11.0   
2975240   93436 2022-03-31  23.8009  1033534.0  1077.59998     3.0   11.0   

          SICCD  DLRET  DLSTCD  
2363     2052.0    NaN     NaN  
2364     

In [ ]:


print(CRSP2018to2022)

         PERMNO       date      RET     SHROUT      ALTPRC  EXCHCD  SHRCD  \
2363      10026 2018-01-31  -8.8191    18678.0   138.44000     3.0   11.0   
2364      10026 2018-02-28  -2.9688    18678.0   134.33000     3.0   11.0   
2365      10026 2018-03-29   1.9951    18697.0   136.56000     3.0   11.0   
2366      10026 2018-04-30   0.6224    18702.0   137.41000     3.0   11.0   
2367      10026 2018-05-31   3.0638    18702.0   141.62000     3.0   11.0   
...         ...        ...      ...        ...         ...     ...    ...   
2975236   93436 2021-11-30   2.7612  1004265.0  1144.76001     3.0   11.0   
2975237   93436 2021-12-31  -7.6855  1033534.0  1056.78003     3.0   11.0   
2975238   93436 2022-01-31 -11.3609  1033534.0   936.71997     3.0   11.0   
2975239   93436 2022-02-28  -7.0768  1033534.0   870.42999     3.0   11.0   
2975240   93436 2022-03-31  23.8009  1033534.0  1077.59998     3.0   11.0   

          SICCD  DLRET  DLSTCD  
2363     2052.0    NaN     NaN  
2364     

In [ ]:
##
#calculate market cap (millions) per row/ calculated column/ new column/ add column
#market cap is total value, shrout is SHaRes OUTstanding, altprc
CRSP2018to2022['market_cap']= abs(CRSP2018to2022['SHROUT']*CRSP2018to2022['ALTPRC'])/1000
#explain this?, inplace not there before
#replacing np.nan (null) rows with 0, will drop later on
CRSP2018to2022['market_cap'].replace(0, np.nan, inplace=True)


print(CRSP2018to2022)

         PERMNO       date      RET     SHROUT      ALTPRC  EXCHCD  SHRCD  \
2363      10026 2018-01-31  -8.8191    18678.0   138.44000     3.0   11.0   
2364      10026 2018-02-28  -2.9688    18678.0   134.33000     3.0   11.0   
2365      10026 2018-03-29   1.9951    18697.0   136.56000     3.0   11.0   
2366      10026 2018-04-30   0.6224    18702.0   137.41000     3.0   11.0   
2367      10026 2018-05-31   3.0638    18702.0   141.62000     3.0   11.0   
...         ...        ...      ...        ...         ...     ...    ...   
2975236   93436 2021-11-30   2.7612  1004265.0  1144.76001     3.0   11.0   
2975237   93436 2021-12-31  -7.6855  1033534.0  1056.78003     3.0   11.0   
2975238   93436 2022-01-31 -11.3609  1033534.0   936.71997     3.0   11.0   
2975239   93436 2022-02-28  -7.0768  1033534.0   870.42999     3.0   11.0   
2975240   93436 2022-03-31  23.8009  1033534.0  1077.59998     3.0   11.0   

          SICCD  DLRET  DLSTCD    market_cap  
2363     2052.0    NaN     N

In [ ]:
#5:56
#add exchange column
#define exchange labels and keep only relevant labels

#1 or 31, traded on NYSE, etc.
CRSP2018to2022['exchange'] = ""
for i in CRSP2018to2022.index:
  if CRSP2018to2022['EXCHCD'][i] in [1, 31]:
    CRSP2018to2022['exchange'][i] = "NYSE"
  elif CRSP2018to2022['EXCHCD'][i] in [2, 32]:
    CRSP2018to2022['exchange'][i] = "AMEX"
  elif CRSP2018to2022['EXCHCD'][i] in [3, 33]:
    CRSP2018to2022['exchange'][i] = "NASDAQ"
  else:
    CRSP2018to2022['exchange'][i] = "Other"
#filter out exchanges of other type ?. ! means inequality. So, keep everything but other
CRSP2018to2022= CRSP2018to2022[CRSP2018to2022['exchange'] != "Other"]

print(CRSP2018to2022)

<ipython-input-16-431f90855819>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CRSP2018to2022['exchange'][i] = "NASDAQ"
<ipython-input-16-431f90855819>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CRSP2018to2022['exchange'][i] = "AMEX"
<ipython-input-16-431f90855819>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CRSP2018to2022['exchange'][i] = "Other"
<ipython-input-16-431f90855819>:9: SettingWithCopyWarning: 
A value is tr

         PERMNO       date      RET     SHROUT      ALTPRC  EXCHCD  SHRCD  \
2363      10026 2018-01-31  -8.8191    18678.0   138.44000     3.0   11.0   
2364      10026 2018-02-28  -2.9688    18678.0   134.33000     3.0   11.0   
2365      10026 2018-03-29   1.9951    18697.0   136.56000     3.0   11.0   
2366      10026 2018-04-30   0.6224    18702.0   137.41000     3.0   11.0   
2367      10026 2018-05-31   3.0638    18702.0   141.62000     3.0   11.0   
...         ...        ...      ...        ...         ...     ...    ...   
2975236   93436 2021-11-30   2.7612  1004265.0  1144.76001     3.0   11.0   
2975237   93436 2021-12-31  -7.6855  1033534.0  1056.78003     3.0   11.0   
2975238   93436 2022-01-31 -11.3609  1033534.0   936.71997     3.0   11.0   
2975239   93436 2022-02-28  -7.0768  1033534.0   870.42999     3.0   11.0   
2975240   93436 2022-03-31  23.8009  1033534.0  1077.59998     3.0   11.0   

          SICCD  DLRET  DLSTCD    market_cap exchange  
2363     2052.0    

In [ ]:
#6:30
#add adjusted return column
#creating new column, all values null
#np.nan is for numbers, as opposed to blank
CRSP2018to2022['ret_adj']= np.nan
#i is variable
for i in CRSP2018to2022.index:
#[column][row],
  if pd.isnull(CRSP2018to2022['DLSTCD'][i]):
    CRSP2018to2022['ret_adj'][i] = CRSP2018to2022['RET'][i]
  elif(pd.notnull(CRSP2018to2022['DLSTCD'][i])) and (pd.notnull(CRSP2018to2022['DLRET'][i])):
    CRSP2018to2022['ret_adj'][i] = CRSP2018to2022['DLSTCD'][i]
#what numbers range mean- index, defined in codebook
  elif(551<=CRSP2018to2022['DLSTCD'][i]<=574) or (CRSP2018to2022['DLSTCD'][i] in [500,520,580,584]):
    CRSP2018to2022['ret_adj'][i] = -30
  else:
    CRSP2018to2022['ret_adj'][i] = -100


print(CRSP2018to2022)

<ipython-input-17-47c0d7a399c7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CRSP2018to2022['ret_adj']= np.nan
<ipython-input-17-47c0d7a399c7>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CRSP2018to2022['ret_adj'][i] = CRSP2018to2022['RET'][i]
<ipython-input-17-47c0d7a399c7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CRSP2018to2022['ret_adj'][i] = -100
<ipython-in

         PERMNO       date      RET     SHROUT      ALTPRC  EXCHCD  SHRCD  \
2363      10026 2018-01-31  -8.8191    18678.0   138.44000     3.0   11.0   
2364      10026 2018-02-28  -2.9688    18678.0   134.33000     3.0   11.0   
2365      10026 2018-03-29   1.9951    18697.0   136.56000     3.0   11.0   
2366      10026 2018-04-30   0.6224    18702.0   137.41000     3.0   11.0   
2367      10026 2018-05-31   3.0638    18702.0   141.62000     3.0   11.0   
...         ...        ...      ...        ...         ...     ...    ...   
2975236   93436 2021-11-30   2.7612  1004265.0  1144.76001     3.0   11.0   
2975237   93436 2021-12-31  -7.6855  1033534.0  1056.78003     3.0   11.0   
2975238   93436 2022-01-31 -11.3609  1033534.0   936.71997     3.0   11.0   
2975239   93436 2022-02-28  -7.0768  1033534.0   870.42999     3.0   11.0   
2975240   93436 2022-03-31  23.8009  1033534.0  1077.59998     3.0   11.0   

          SICCD  DLRET  DLSTCD    market_cap exchange   ret_adj  
2363     

In [ ]:
#6:40
#drop unneeded collumns, neater dataframe

#no need for pandas, only sequel
#CRSP2018to2022= CRSP2018to2022.reset_index()

##HARSHA; ALSO DROPPED INDEX
#axis 0 is certain row (implicit), axis 1 is certain columns (have to make explicit)
CRSP2018to2022.drop(['SHRCD', 'DLRET', 'DLSTCD'], inplace=True, axis=1)

print(CRSP2018to2022)
print(CRSP2018to2022.columns)


         PERMNO       date      RET     SHROUT      ALTPRC  EXCHCD   SICCD  \
2363      10026 2018-01-31  -8.8191    18678.0   138.44000     3.0  2052.0   
2364      10026 2018-02-28  -2.9688    18678.0   134.33000     3.0  2052.0   
2365      10026 2018-03-29   1.9951    18697.0   136.56000     3.0  2052.0   
2366      10026 2018-04-30   0.6224    18702.0   137.41000     3.0  2052.0   
2367      10026 2018-05-31   3.0638    18702.0   141.62000     3.0  2052.0   
...         ...        ...      ...        ...         ...     ...     ...   
2975236   93436 2021-11-30   2.7612  1004265.0  1144.76001     3.0  9999.0   
2975237   93436 2021-12-31  -7.6855  1033534.0  1056.78003     3.0  9999.0   
2975238   93436 2022-01-31 -11.3609  1033534.0   936.71997     3.0  9999.0   
2975239   93436 2022-02-28  -7.0768  1033534.0   870.42999     3.0  9999.0   
2975240   93436 2022-03-31  23.8009  1033534.0  1077.59998     3.0  9999.0   

           market_cap exchange   ret_adj  
2363     2.585782e+0

clean CCM data

In [ ]:

file = "/content/drive/MyDrive/Exeter google drive /internships jobs/FIMA '23/Fama-french 5 factor/ccmfund1990to2022.csv"
ccmfund1990to2022 = pd.read_csv(file)
print(ccmfund1990to2022)

<ipython-input-19-11549abe2e8c>:2: DtypeWarning: Columns (2,17,19,24,33,37,952,953,955,956,957,962,976,977,982) have mixed types. Specify dtype option on import or set low_memory=False.
  ccmfund1990to2022 = pd.read_csv(file)


         GVKEY LINKPRIM LIID LINKTYPE  LPERMNO  LPERMCO    LINKDT LINKENDDT  \
0         1004        P    1       LU    54594    20000  19720424         E   
1         1004        P    1       LU    54594    20000  19720424         E   
2         1004        P    1       LU    54594    20000  19720424         E   
3         1004        P    1       LU    54594    20000  19720424         E   
4         1004        P    1       LU    54594    20000  19720424         E   
...        ...      ...  ...      ...      ...      ...       ...       ...   
209121  345980        P    1       LC    20333    57666  20201216         E   
209122  345980        P    1       LC    20333    57666  20201216         E   
209123  347007        P    1       LC    15533    55364  20210310         E   
209124  347085        P    1       LC    21069    58208  20210401         E   
209125  349530        P    1       LC    17324    56274  20210709         E   

        datadate   fyear  ... priusa     sic spcind

In [ ]:
#only keeping these relevant variables
ccmfund1990to2022= ccmfund1990to2022[["GVKEY", "LPERMNO", "datadate", "LINKTYPE", "LINKENDDT", "seq", "ceq", "at", "lt", "txditc","txdb", "itcb", "pstkrv", "pstkl", "pstk", "indfmt", "datafmt", "revt","cogs","xint", "xsga"]]
#drop duplicates (instead of slect distinct)
ccmfund1990to2022.drop_duplicates(inplace=True)

In [ ]:
print(ccmfund1990to2022.columns)

Index(['GVKEY', 'LPERMNO', 'datadate', 'LINKTYPE', 'LINKENDDT', 'seq', 'ceq',
       'at', 'lt', 'txditc', 'txdb', 'itcb', 'pstkrv', 'pstkl', 'pstk',
       'indfmt', 'datafmt', 'revt', 'cogs', 'xint', 'xsga'],
      dtype='object')


In [ ]:

print(ccmfund1990to2022)

         GVKEY  LPERMNO  datadate LINKTYPE LINKENDDT       seq       ceq  \
0         1004    54594  19910531       LU         E   193.778   193.778   
1         1004    54594  19920531       LU         E   196.737   196.737   
2         1004    54594  19930531       LU         E   189.216   189.216   
3         1004    54594  19940531       LU         E   189.488   189.488   
4         1004    54594  19950531       LU         E   197.119   197.119   
...        ...      ...       ...      ...       ...       ...       ...   
209121  345980    20333  20201231       LC         E  1027.000  1027.000   
209122  345980    20333  20211231       LC         E   818.000   818.000   
209123  347007    15533  20211231       LC         E  -242.173  -242.173   
209124  347085    21069  20220228       LC         E   139.431   139.431   
209125  349530    17324  20220228       LC         E    64.231    64.231   

              at        lt  txditc  ...  itcb  pstkrv  pstkl  pstk  indfmt  \
0        

In [ ]:
#7:57

#ccmfund1990to2022['GVKEY']= pd.to_numeric(ccmfund1990to2022['GVKEY'], errors= 'coerce')
#ccmfund1990to2022['LPERMNO']= pd.to_numeric(ccmfund1990to2022['LPERMNO'], errors= 'coerce')
#ccmfund1990to2022['datadate']= pd.to_datetime(ccmfund1990to2022['datadate'], format= '%Y%m%d')
# ccmfund1990to2022['LINKTYPE']= ccmfund1990to2022['LINKTYPE'].apply(str)
# ccmfund1990to2022['LINKENDDT']= pd.to_datetime(ccmfund1990to2022['LINKENDDT'], format= '%Y%m%d', errors='coerce')
# ccmfund1990to2022['seq']= pd.to_numeric(ccmfund1990to2022['seq'], errors= 'coerce')
# ccmfund1990to2022['ceq']= pd.to_numeric(ccmfund1990to2022['ceq'], errors= 'coerce')
# ccmfund1990to2022['at']= pd.to_numeric(ccmfund1990to2022['at'], errors= 'coerce')
# ccmfund1990to2022['lt']= pd.to_numeric(ccmfund1990to2022['lt'], errors= 'coerce')
# ccmfund1990to2022['txditc']= pd.to_numeric(ccmfund1990to2022['txditc'], errors= 'coerce')
# ccmfund1990to2022['txdb']= pd.to_numeric(ccmfund1990to2022['txdb'], errors= 'coerce')
# ccmfund1990to2022['itcb']= pd.to_numeric(ccmfund1990to2022['itcb'], errors= 'coerce')
# ccmfund1990to2022['pstkrv']= pd.to_numeric(ccmfund1990to2022['pstkrv'], errors= 'coerce')
# ccmfund1990to2022['pstkl']= pd.to_numeric(ccmfund1990to2022['pstkl'], errors= 'coerce')
# ccmfund1990to2022['pstk']= pd.to_numeric(ccmfund1990to2022['pstk'], errors= 'coerce')

for col in ['datadate', 'LINKENDDT']:
    ccmfund1990to2022[col] = pd.to_datetime(ccmfund1990to2022[col], format = '%Y%m%d', errors = 'coerce')
for col in ['LINKTYPE', 'indfmt', 'datafmt']:
    ccmfund1990to2022[col] = ccmfund1990to2022[col].apply(str)
for col in ['GVKEY', 'LPERMNO', 'seq', 'ceq', 'at', 'lt', 'txditc', 'txdb',
            'itcb', 'pstkrv', 'pstkl', 'pstk', 'revt', 'cogs', 'xint', 'xsga']:
    ccmfund1990to2022[col] = pd.to_numeric(ccmfund1990to2022[col], errors = 'coerce')
for col in ['datadate', 'LINKENDDT']:
    ccmfund1990to2022[col] = pd.to_datetime(ccmfund1990to2022[col], format = '%Y%m%d', errors = 'coerce')
for col in ['LINKTYPE', 'indfmt', 'datafmt']:
    ccmfund1990to2022[col] = ccmfund1990to2022[col].apply(str)

print(ccmfund1990to2022)



         GVKEY  LPERMNO   datadate LINKTYPE LINKENDDT       seq       ceq  \
0         1004    54594 1991-05-31       LU       NaT   193.778   193.778   
1         1004    54594 1992-05-31       LU       NaT   196.737   196.737   
2         1004    54594 1993-05-31       LU       NaT   189.216   189.216   
3         1004    54594 1994-05-31       LU       NaT   189.488   189.488   
4         1004    54594 1995-05-31       LU       NaT   197.119   197.119   
...        ...      ...        ...      ...       ...       ...       ...   
209121  345980    20333 2020-12-31       LC       NaT  1027.000  1027.000   
209122  345980    20333 2021-12-31       LC       NaT   818.000   818.000   
209123  347007    15533 2021-12-31       LC       NaT  -242.173  -242.173   
209124  347085    21069 2022-02-28       LC       NaT   139.431   139.431   
209125  349530    17324 2022-02-28       LC       NaT    64.231    64.231   

              at        lt  txditc  ...  itcb  pstkrv  pstkl  pstk  indfmt 

In [ ]:
ccmfund1990to2022['datadate'] = pd.to_datetime(ccmfund1990to2022['datadate'], format='%Y%m%d')
ccmfund1990to2022['LINKENDDT'] = pd.to_datetime(ccmfund1990to2022['LINKENDDT'], format='%Y%m%d', errors= 'coerce')

In [ ]:

print(ccmfund1990to2022)

         GVKEY  LPERMNO   datadate LINKTYPE LINKENDDT       seq       ceq  \
0         1004    54594 1991-05-31       LU       NaT   193.778   193.778   
1         1004    54594 1992-05-31       LU       NaT   196.737   196.737   
2         1004    54594 1993-05-31       LU       NaT   189.216   189.216   
3         1004    54594 1994-05-31       LU       NaT   189.488   189.488   
4         1004    54594 1995-05-31       LU       NaT   197.119   197.119   
...        ...      ...        ...      ...       ...       ...       ...   
209121  345980    20333 2020-12-31       LC       NaT  1027.000  1027.000   
209122  345980    20333 2021-12-31       LC       NaT   818.000   818.000   
209123  347007    15533 2021-12-31       LC       NaT  -242.173  -242.173   
209124  347085    21069 2022-02-28       LC       NaT   139.431   139.431   
209125  349530    17324 2022-02-28       LC       NaT    64.231    64.231   

              at        lt  txditc  ...  itcb  pstkrv  pstkl  pstk  indfmt 

In [ ]:

#3:17 5 factor
ccmfund1990to2022['revt']= pd.to_numeric(ccmfund1990to2022['revt'], errors= 'coerce')
#cost of goods sold
ccmfund1990to2022['cogs']= pd.to_numeric(ccmfund1990to2022['cogs'], errors= 'coerce')
#intrest expense
ccmfund1990to2022['xint']= pd.to_numeric(ccmfund1990to2022['xint'], errors= 'coerce')
#selling, general, admin expense
ccmfund1990to2022['xsga']= pd.to_numeric(ccmfund1990to2022['xsga'], errors= 'coerce')


ccmfund1990to2022['indfmt']= ccmfund1990to2022['indfmt'].apply(str)
ccmfund1990to2022['datafmt']= ccmfund1990to2022['datafmt'].apply(str)

In [ ]:
print(ccmfund1990to2022)

         GVKEY  LPERMNO   datadate LINKTYPE LINKENDDT       seq       ceq  \
0         1004    54594 1991-05-31       LU       NaT   193.778   193.778   
1         1004    54594 1992-05-31       LU       NaT   196.737   196.737   
2         1004    54594 1993-05-31       LU       NaT   189.216   189.216   
3         1004    54594 1994-05-31       LU       NaT   189.488   189.488   
4         1004    54594 1995-05-31       LU       NaT   197.119   197.119   
...        ...      ...        ...      ...       ...       ...       ...   
209121  345980    20333 2020-12-31       LC       NaT  1027.000  1027.000   
209122  345980    20333 2021-12-31       LC       NaT   818.000   818.000   
209123  347007    15533 2021-12-31       LC       NaT  -242.173  -242.173   
209124  347085    21069 2022-02-28       LC       NaT   139.431   139.431   
209125  349530    17324 2022-02-28       LC       NaT    64.231    64.231   

              at        lt  txditc  ...  itcb  pstkrv  pstkl  pstk  indfmt 

In [ ]:
#8:04

#only data rows after 2017 (two years before 2019 to have assets from 2018/2017)
ccmfund2018to2022= ccmfund1990to2022[ccmfund1990to2022['datadate']>= "2017-01-01 00:00:00"]


#only keep data in correct format
ccmfund2018to2022= ccmfund2018to2022[ccmfund2018to2022['indfmt']=="INDL"]
ccmfund2018to2022= ccmfund2018to2022[ccmfund2018to2022['datafmt']=="STD"]

#only keep valid links
ccmfund2018to2022= ccmfund2018to2022[(ccmfund2018to2022['LINKTYPE']=="LU")| (ccmfund2018to2022['LINKTYPE']== "LC")]

#only keep links active at datadate
ccmfund2018to2022= ccmfund2018to2022[(ccmfund2018to2022['datadate']<= ccmfund2018to2022['LINKENDDT'])| pd.isnull(ccmfund2018to2022['LINKENDDT'])]


In [ ]:
# prompt: use pandas to only take dates from column 'date' after 2018

#CRSP2018to2022= CRSPMonthly1990Through2022[CRSPMonthly1990Through2022['date']>= "2018-01-01"]

In [ ]:
print(ccmfund2018to2022)

         GVKEY  LPERMNO   datadate LINKTYPE LINKENDDT       seq       ceq  \
26        1004    54594 2017-05-31       LU       NaT   914.200   914.200   
27        1004    54594 2018-05-31       LU       NaT   936.300   936.300   
28        1004    54594 2019-05-31       LU       NaT   905.900   905.900   
29        1004    54594 2020-05-31       LU       NaT   902.600   902.600   
30        1004    54594 2021-05-31       LU       NaT   974.400   974.400   
...        ...      ...        ...      ...       ...       ...       ...   
209121  345980    20333 2020-12-31       LC       NaT  1027.000  1027.000   
209122  345980    20333 2021-12-31       LC       NaT   818.000   818.000   
209123  347007    15533 2021-12-31       LC       NaT  -242.173  -242.173   
209124  347085    21069 2022-02-28       LC       NaT   139.431   139.431   
209125  349530    17324 2022-02-28       LC       NaT    64.231    64.231   

              at        lt  txditc  ...  itcb  pstkrv  pstkl  pstk  indfmt 

In [ ]:
#8:42

#book value calculated column

#?replace or vs coalesce
#ccmfund1990to2022['book_value']= abs(ccmfund1990to2022['seq']*CRSP2018to2022['ALTPRC'])
#df['A'].combine_first(df['B'])
#df['c'] = np.where(df["a"].isnull(), df["b"], df["a"] )
ccmfund2018to2022 = ccmfund2018to2022.rename(columns={'GVKEY': 'gvkey', 'LPERMNO': 'permno'})


#variable a
##ccmfund1990to2022['w']= ""
#ccmfund1990to2022['w']= ccmfund1990to2022['seq'].combine_first(ccmfund1990to2022['ceq']+ ccmfund1990to2022['pstk']).combine_first(ccmfund1990to2022['at']- ccmfund1990to2022['it'])
#variable b
#ccmfund1990to2022['y']= ""
#ccmfund1990to2022['y']= ccmfund1990to2022['txditc'].combine_first(ccmfund1990to2022['txdb']+ ccmfund1990to2022['itcb']).combine_first(0)
#if ccmfund1990to2022['y']= null:
  #ccmfund1990to2022['y']= 0


#important for coalesce function: fillna in this format
#ccmfund1990to2022['book_value']= ccmfund1990to2022['seq'].fillna(ccmfund1990to2022['ceq']+ ccmfund1990to2022['pstk']).fillna(ccmfund1990to2022['at']- ccmfund1990to2022['it']) + ccmfund1990to2022['txditc'].fillna(ccmfund1990to2022['txdb']+ ccmfund1990to2022['itcb']).fillna(0) -
ccmfund2018to2022['book_value'] = (ccmfund2018to2022['seq'].fillna(ccmfund2018to2022['ceq'] + ccmfund2018to2022['pstk']).fillna(ccmfund2018to2022['at'] - ccmfund2018to2022['lt'])
            + ccmfund2018to2022['txditc'].fillna(ccmfund2018to2022['txdb'] + ccmfund2018to2022['itcb']).fillna(0)
            + ccmfund2018to2022['pstkrv'].fillna(ccmfund2018to2022['pstkl']).fillna(ccmfund2018to2022['pstk']).fillna(0))


ccmfund2018to2022['OP']= (ccmfund2018to2022['revt'] - ccmfund2018to2022['cogs'] - ccmfund2018to2022['xint'] - ccmfund2018to2022['xsga'] /
                         + ccmfund2018to2022['seq'].fillna(ccmfund2018to2022['ceq'] + ccmfund2018to2022['pstk']).fillna(ccmfund2018to2022['at'] - ccmfund2018to2022['lt'])
                         + ccmfund2018to2022['txditc'].fillna(ccmfund2018to2022['txdb'] + ccmfund2018to2022['itcb']).fillna(0)
                         + ccmfund2018to2022['pstkrv'].fillna(ccmfund2018to2022['pstkl']).fillna(ccmfund2018to2022['pstk']).fillna(0))


#ccmfund1990to2022['revt']-ccmfund1990to2022['cogs']-ccmfund1990to2022['xint']-ccmfund1990to2022['xsga']/ ccmfund1990to2022['seq'].fillna(ccmfund1990to2022['ceq']+ ccmfund1990to2022['pstk']).fillna(ccmfund1990to2022['at']- ccmfund1990to2022['1t']) + ccmfund1990to2022['txditc'].fillna(ccmfund1990to2022['txdb']+ ccmfund1990to2022['itcb']).fillna(0) - ccmfund1990to2022['pstkrv'].fillna(ccmfund1990to2022['pstkl']).fillna(ccmfund1990to2022['pstk']).fillna(0)



#explain this?, inplace not there before
#replacing np.nan (null) rows with 0, will drop later on
ccmfund2018to2022['book_value'].replace(0, np.nan, inplace=True)
ccmfund2018to2022['OP'].replace(0, np.nan, inplace=True)


In [ ]:
print(ccmfund2018to2022)

         gvkey  permno   datadate LINKTYPE LINKENDDT       seq       ceq  \
26        1004   54594 2017-05-31       LU       NaT   914.200   914.200   
27        1004   54594 2018-05-31       LU       NaT   936.300   936.300   
28        1004   54594 2019-05-31       LU       NaT   905.900   905.900   
29        1004   54594 2020-05-31       LU       NaT   902.600   902.600   
30        1004   54594 2021-05-31       LU       NaT   974.400   974.400   
...        ...     ...        ...      ...       ...       ...       ...   
209121  345980   20333 2020-12-31       LC       NaT  1027.000  1027.000   
209122  345980   20333 2021-12-31       LC       NaT   818.000   818.000   
209123  347007   15533 2021-12-31       LC       NaT  -242.173  -242.173   
209124  347085   21069 2022-02-28       LC       NaT   139.431   139.431   
209125  349530   17324 2022-02-28       LC       NaT    64.231    64.231   

              at        lt  txditc  ...  pstkl  pstk  indfmt  datafmt  \
26      1504.1

In [ ]:
print(ccmfund2018to2022.columns)
##harsha reduced number of columns, also index is different

Index(['gvkey', 'permno', 'datadate', 'LINKTYPE', 'LINKENDDT', 'seq', 'ceq',
       'at', 'lt', 'txditc', 'txdb', 'itcb', 'pstkrv', 'pstkl', 'pstk',
       'indfmt', 'datafmt', 'revt', 'cogs', 'xint', 'xsga', 'book_value',
       'OP'],
      dtype='object')


In [ ]:

ccmfund2018to2022 = ccmfund2018to2022.reset_index()
ccmfund2018to2022.drop('index', inplace=True, axis=1) # no index column
print(ccmfund2018to2022)

        gvkey  permno   datadate LINKTYPE LINKENDDT       seq       ceq  \
0        1004   54594 2017-05-31       LU       NaT   914.200   914.200   
1        1004   54594 2018-05-31       LU       NaT   936.300   936.300   
2        1004   54594 2019-05-31       LU       NaT   905.900   905.900   
3        1004   54594 2020-05-31       LU       NaT   902.600   902.600   
4        1004   54594 2021-05-31       LU       NaT   974.400   974.400   
...       ...     ...        ...      ...       ...       ...       ...   
27656  345980   20333 2020-12-31       LC       NaT  1027.000  1027.000   
27657  345980   20333 2021-12-31       LC       NaT   818.000   818.000   
27658  347007   15533 2021-12-31       LC       NaT  -242.173  -242.173   
27659  347085   21069 2022-02-28       LC       NaT   139.431   139.431   
27660  349530   17324 2022-02-28       LC       NaT    64.231    64.231   

             at        lt  txditc  ...  pstkl  pstk  indfmt  datafmt  \
0      1504.100   589.900  

In [ ]:
#8:57

#keep only the last observation of each year
ccmfund2018to2022['datadate']= pd.to_datetime(ccmfund2018to2022['datadate'])
ccmfund2018to2022['year']= ccmfund2018to2022['datadate'].dt.year
ccmfund2018to2022= ccmfund2018to2022.sort_values(by= ['permno', "datadate"], ascending= True)

#defining function, drop what meets if
toDrop=[]
#drop june observation
#for i in range(len(ccmfund2018to2022)-1):
for i in range(len(ccmfund2018to2022) - 1):
 if (ccmfund2018to2022['permno'][i] == ccmfund2018to2022['permno'][i+1]) and (ccmfund2018to2022['year'][i] == ccmfund2018to2022['year'][i+1]):
  toDrop.append(i)
ccmfund2018to2022.drop(toDrop, inplace = True, axis= 0)
#ccmfund1990to2022.drop('index', inplace = True, axis= 1)

print(ccmfund2018to2022)
###????
#ccmfund1990to2022.drop(['SHRCD', 'DLRET', 'DLSTCD'], inplace=True, axis=1)


        gvkey  permno   datadate LINKTYPE LINKENDDT        seq        ceq  \
4493    12825   10026 2017-09-30       LU       NaT    682.322    682.322   
4494    12825   10026 2018-09-30       LU       NaT    759.091    759.091   
4495    12825   10026 2019-09-30       LU       NaT    833.751    833.751   
4496    12825   10026 2020-09-30       LU       NaT    809.498    809.498   
4497    12825   10026 2021-09-30       LU       NaT    845.654    845.654   
...       ...     ...        ...      ...       ...        ...        ...   
25655  184996   93436 2017-12-31       LC       NaT   4237.312   4237.242   
25656  184996   93436 2018-12-31       LC       NaT   4923.243   4923.243   
25657  184996   93436 2019-12-31       LC       NaT   6618.000   6618.000   
25658  184996   93436 2020-12-31       LC       NaT  22225.000  22225.000   
25659  184996   93436 2021-12-31       LC       NaT  30189.000  30189.000   

              at         lt   txditc  ...  pstk  indfmt  datafmt       revt

In [ ]:
# prompt: only keep df columns 'f', 'd'

ccmfund2018to2022 = ccmfund2018to2022[['gvkey', 'permno', 'datadate', 'at','book_value','OP','year']]

print(ccmfund2018to2022)

        gvkey  permno   datadate         at  book_value            OP  year
4493    12825   10026 2017-09-30    867.228     745.027           NaN  2017
4494    12825   10026 2018-09-30    932.013     811.413           NaN  2018
4495    12825   10026 2019-09-30   1019.339     895.671           NaN  2019
4496    12825   10026 2020-09-30   1056.553     873.911           NaN  2020
4497    12825   10026 2021-09-30   1122.219     907.232           NaN  2021
...       ...     ...        ...        ...         ...           ...   ...
25655  184996   93436 2017-12-31  28655.372    4237.312   3261.424138  2017
25656  184996   93436 2018-12-31  29739.614    4923.243   5210.963636  2018
25657  184996   93436 2019-12-31  34309.000    6618.000   5459.397250  2019
25658  184996   93436 2020-12-31  52148.000   22376.000   8644.789471  2020
25659  184996   93436 2021-12-31  62131.000   30213.000  16192.764484  2021

[27650 rows x 7 columns]


In [ ]:
#clean
ccmfund2018to2022.replace([np.inf, -np.inf], np.nan, inplace= True)
ccmfund2018to2022.dropna(inplace=True)


<ipython-input-37-de64ac08747e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ccmfund2018to2022.replace([np.inf, -np.inf], np.nan, inplace= True)
<ipython-input-37-de64ac08747e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ccmfund2018to2022.dropna(inplace=True)


In [ ]:
print(ccmfund2018to2022)

        gvkey  permno   datadate         at  book_value            OP  year
4174    12096   10028 2017-12-31     13.315       7.756      9.930800  2017
4175    12096   10028 2018-12-31     13.018       8.413      8.638047  2018
4176    12096   10028 2019-12-31     27.003      11.194     14.732460  2019
4177    12096   10028 2020-12-31     40.579      17.578     21.564201  2020
4178    12096   10028 2021-12-31     59.267      27.627     29.764186  2021
...       ...     ...        ...        ...         ...           ...   ...
25655  184996   93436 2017-12-31  28655.372    4237.312   3261.424138  2017
25656  184996   93436 2018-12-31  29739.614    4923.243   5210.963636  2018
25657  184996   93436 2019-12-31  34309.000    6618.000   5459.397250  2019
25658  184996   93436 2020-12-31  52148.000   22376.000   8644.789471  2020
25659  184996   93436 2021-12-31  62131.000   30213.000  16192.764484  2021

[18093 rows x 7 columns]


In [ ]:
# Reindex
ccmfund2018to2022 = ccmfund2018to2022.reset_index()
ccmfund2018to2022.drop('index', inplace=True, axis=1) # no index column
print(ccmfund2018to2022)

        gvkey  permno   datadate         at  book_value            OP  year
0       12096   10028 2017-12-31     13.315       7.756      9.930800  2017
1       12096   10028 2018-12-31     13.018       8.413      8.638047  2018
2       12096   10028 2019-12-31     27.003      11.194     14.732460  2019
3       12096   10028 2020-12-31     40.579      17.578     21.564201  2020
4       12096   10028 2021-12-31     59.267      27.627     29.764186  2021
...       ...     ...        ...        ...         ...           ...   ...
18088  184996   93436 2017-12-31  28655.372    4237.312   3261.424138  2017
18089  184996   93436 2018-12-31  29739.614    4923.243   5210.963636  2018
18090  184996   93436 2019-12-31  34309.000    6618.000   5459.397250  2019
18091  184996   93436 2020-12-31  52148.000   22376.000   8644.789471  2020
18092  184996   93436 2021-12-31  62131.000   30213.000  16192.764484  2021

[18093 rows x 7 columns]


In [ ]:
#3:56 FF

# ccmfund2018to2022.reset_index
# #calculated column for INV

ccmfund2018to2022['inv']= ""

for i in range(1, len(ccmfund2018to2022)):
    if ccmfund2018to2022['permno'][i] == ccmfund2018to2022['permno'][i-1]:
        ccmfund2018to2022['inv'][i] = ccmfund2018to2022['at'][i] / ccmfund2018to2022['at'][i-1]


# ccmfund2018to2022['inv']= ""
# #len(df)= rows in dataframe
# for i in ccmfund2018to2022.index:
#   if ccmfund2018to2022['permno'][i]== ccmfund2018to2022['permno'][i-1]:
#     #are you getting year-1/ year-2?, is reference date alr 1 year prior?
#     ccmfund2018to2022['inv'][i]= ccmfund2018to2022['at'][i]/ ccmfund2018to2022['at'][i-1]

<ipython-input-40-933ee80521ed>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ccmfund2018to2022['inv'][i] = ccmfund2018to2022['at'][i] / ccmfund2018to2022['at'][i-1]


In [ ]:
print(ccmfund2018to2022)

        gvkey  permno   datadate         at  book_value            OP  year  \
0       12096   10028 2017-12-31     13.315       7.756      9.930800  2017   
1       12096   10028 2018-12-31     13.018       8.413      8.638047  2018   
2       12096   10028 2019-12-31     27.003      11.194     14.732460  2019   
3       12096   10028 2020-12-31     40.579      17.578     21.564201  2020   
4       12096   10028 2021-12-31     59.267      27.627     29.764186  2021   
...       ...     ...        ...        ...         ...           ...   ...   
18088  184996   93436 2017-12-31  28655.372    4237.312   3261.424138  2017   
18089  184996   93436 2018-12-31  29739.614    4923.243   5210.963636  2018   
18090  184996   93436 2019-12-31  34309.000    6618.000   5459.397250  2019   
18091  184996   93436 2020-12-31  52148.000   22376.000   8644.789471  2020   
18092  184996   93436 2021-12-31  62131.000   30213.000  16192.764484  2021   

            inv  
0                
1      0.977694

In [ ]:
#9:15
#drop missing/infinite values

ccmfund2018to2022.replace([np.inf, -np.inf], np.nan, inplace=True)
ccmfund2018to2022= ccmfund2018to2022.dropna()

print(ccmfund2018to2022)

        gvkey  permno   datadate         at  book_value            OP  year  \
0       12096   10028 2017-12-31     13.315       7.756      9.930800  2017   
1       12096   10028 2018-12-31     13.018       8.413      8.638047  2018   
2       12096   10028 2019-12-31     27.003      11.194     14.732460  2019   
3       12096   10028 2020-12-31     40.579      17.578     21.564201  2020   
4       12096   10028 2021-12-31     59.267      27.627     29.764186  2021   
...       ...     ...        ...        ...         ...           ...   ...   
18088  184996   93436 2017-12-31  28655.372    4237.312   3261.424138  2017   
18089  184996   93436 2018-12-31  29739.614    4923.243   5210.963636  2018   
18090  184996   93436 2019-12-31  34309.000    6618.000   5459.397250  2019   
18091  184996   93436 2020-12-31  52148.000   22376.000   8644.789471  2020   
18092  184996   93436 2021-12-31  62131.000   30213.000  16192.764484  2021   

            inv  
0                
1      0.977694

In [ ]:
#9:21
#add reference date column (for matching; june 1 6/1 of next calander year)
ccmfund2018to2022['year']= pd.to_numeric(ccmfund2018to2022['year'], errors= 'coerce')
ccmfund2018to2022['reference_date']= ccmfund2018to2022['year'] +1
ccmfund2018to2022['reference_date']= ccmfund2018to2022['reference_date'].astype(str)
#add year to string
#difference btw reference date and year?- 12/23 vs 6/24
ccmfund2018to2022['reference_date']+= "-06-01"
ccmfund2018to2022['reference_date']= pd.to_datetime(ccmfund2018to2022['reference_date'])
ccmfund2018to2022.drop('year', inplace= True, axis=1 )

print(ccmfund2018to2022)
print(ccmfund2018to2022.columns)

        gvkey  permno   datadate         at  book_value            OP  \
0       12096   10028 2017-12-31     13.315       7.756      9.930800   
1       12096   10028 2018-12-31     13.018       8.413      8.638047   
2       12096   10028 2019-12-31     27.003      11.194     14.732460   
3       12096   10028 2020-12-31     40.579      17.578     21.564201   
4       12096   10028 2021-12-31     59.267      27.627     29.764186   
...       ...     ...        ...        ...         ...           ...   
18088  184996   93436 2017-12-31  28655.372    4237.312   3261.424138   
18089  184996   93436 2018-12-31  29739.614    4923.243   5210.963636   
18090  184996   93436 2019-12-31  34309.000    6618.000   5459.397250   
18091  184996   93436 2020-12-31  52148.000   22376.000   8644.789471   
18092  184996   93436 2021-12-31  62131.000   30213.000  16192.764484   

            inv reference_date  
0                   2018-06-01  
1      0.977694     2019-06-01  
2      2.074282     2020

CRSP #CRSP2018to2022 AND COMPUSTAT #ccmfund1990to2022

In [ ]:
print(CRSP2018to2022)
print(CRSP2018to2022.columns)

         PERMNO       date      RET     SHROUT      ALTPRC  EXCHCD   SICCD  \
2363      10026 2018-01-31  -8.8191    18678.0   138.44000     3.0  2052.0   
2364      10026 2018-02-28  -2.9688    18678.0   134.33000     3.0  2052.0   
2365      10026 2018-03-29   1.9951    18697.0   136.56000     3.0  2052.0   
2366      10026 2018-04-30   0.6224    18702.0   137.41000     3.0  2052.0   
2367      10026 2018-05-31   3.0638    18702.0   141.62000     3.0  2052.0   
...         ...        ...      ...        ...         ...     ...     ...   
2975236   93436 2021-11-30   2.7612  1004265.0  1144.76001     3.0  9999.0   
2975237   93436 2021-12-31  -7.6855  1033534.0  1056.78003     3.0  9999.0   
2975238   93436 2022-01-31 -11.3609  1033534.0   936.71997     3.0  9999.0   
2975239   93436 2022-02-28  -7.0768  1033534.0   870.42999     3.0  9999.0   
2975240   93436 2022-03-31  23.8009  1033534.0  1077.59998     3.0  9999.0   

           market_cap exchange   ret_adj  
2363     2.585782e+0

In [ ]:
#9:44
#parse relevant CRSP variables and store as stocks dataframe

#only keeping these relevant variables
#substitute for select distinct
#harsha change; ret adj is ret,
stonks2018To2022= CRSP2018to2022[["PERMNO", "date", "ret_adj","market_cap", 'exchange']]
#drop duplicates (instead of select distinct)
stonks2018To2022.drop_duplicates(inplace=True)

#drop infinite/missing values
stonks2018To2022.replace([np.inf, -np.inf], np.nan, inplace=True)
stonks2018To2022= stonks2018To2022.dropna()


<ipython-input-45-71dcb8e8c9de>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stonks2018To2022.drop_duplicates(inplace=True)
<ipython-input-45-71dcb8e8c9de>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stonks2018To2022.replace([np.inf, -np.inf], np.nan, inplace=True)


In [ ]:

print(stonks2018To2022)

         PERMNO       date   ret_adj    market_cap exchange
2363      10026 2018-01-31   -8.8191  2.585782e+03   NASDAQ
2364      10026 2018-02-28   -2.9688  2.509016e+03   NASDAQ
2365      10026 2018-03-29    1.9951  2.553262e+03   NASDAQ
2366      10026 2018-04-30    0.6224  2.569842e+03   NASDAQ
2367      10026 2018-05-31    3.0638  2.648577e+03   NASDAQ
...         ...        ...       ...           ...      ...
2975236   93436 2021-11-30    2.7612  1.149642e+06   NASDAQ
2975237   93436 2021-12-31   -7.6855  1.092218e+06   NASDAQ
2975238   93436 2022-01-31  -11.3609  9.681319e+05   NASDAQ
2975239   93436 2022-02-28   -7.0768  8.996190e+05   NASDAQ
2975240   93436 2022-03-31 -100.0000  1.113736e+06   NASDAQ

[191617 rows x 5 columns]


In [ ]:
#9:55
#add reference date to merdge stonks and cmmfund

stonks2018To2022['date']= pd.to_datetime(stonks2018To2022['date'])

stonks2018To2022['reference_date']= ""
for i in stonks2018To2022.index:
  if stonks2018To2022['date'][i].month<6:

#stonks2018To2022['date']= stonks2018To2022['date'].astype(str)
    stonks2018To2022['reference_date'][i]= (stonks2018To2022['date'][i].year-1)
  else:
    stonks2018To2022['reference_date'][i]= (stonks2018To2022['date'][i].year)

stonks2018To2022['reference_date']= stonks2018To2022['reference_date'].astype(str) +"-06-01"

stonks2018To2022['reference_date']= pd.to_datetime(stonks2018To2022['reference_date'])


<ipython-input-47-db2c36a1e7be>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stonks2018To2022['reference_date'][i]= (stonks2018To2022['date'][i].year-1)
<ipython-input-47-db2c36a1e7be>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stonks2018To2022['reference_date'][i]= (stonks2018To2022['date'][i].year)


In [ ]:

print(stonks2018To2022)

         PERMNO       date   ret_adj    market_cap exchange reference_date
2363      10026 2018-01-31   -8.8191  2.585782e+03   NASDAQ     2017-06-01
2364      10026 2018-02-28   -2.9688  2.509016e+03   NASDAQ     2017-06-01
2365      10026 2018-03-29    1.9951  2.553262e+03   NASDAQ     2017-06-01
2366      10026 2018-04-30    0.6224  2.569842e+03   NASDAQ     2017-06-01
2367      10026 2018-05-31    3.0638  2.648577e+03   NASDAQ     2017-06-01
...         ...        ...       ...           ...      ...            ...
2975236   93436 2021-11-30    2.7612  1.149642e+06   NASDAQ     2021-06-01
2975237   93436 2021-12-31   -7.6855  1.092218e+06   NASDAQ     2021-06-01
2975238   93436 2022-01-31  -11.3609  9.681319e+05   NASDAQ     2021-06-01
2975239   93436 2022-02-28   -7.0768  8.996190e+05   NASDAQ     2021-06-01
2975240   93436 2022-03-31 -100.0000  1.113736e+06   NASDAQ     2021-06-01

[191617 rows x 6 columns]


In [ ]:

stonks2018To2022.rename(columns={'PERMNO':'permno', 'ret_adj':'ret'}, inplace=True)
print(stonks2018To2022)


         permno       date       ret    market_cap exchange reference_date
2363      10026 2018-01-31   -8.8191  2.585782e+03   NASDAQ     2017-06-01
2364      10026 2018-02-28   -2.9688  2.509016e+03   NASDAQ     2017-06-01
2365      10026 2018-03-29    1.9951  2.553262e+03   NASDAQ     2017-06-01
2366      10026 2018-04-30    0.6224  2.569842e+03   NASDAQ     2017-06-01
2367      10026 2018-05-31    3.0638  2.648577e+03   NASDAQ     2017-06-01
...         ...        ...       ...           ...      ...            ...
2975236   93436 2021-11-30    2.7612  1.149642e+06   NASDAQ     2021-06-01
2975237   93436 2021-12-31   -7.6855  1.092218e+06   NASDAQ     2021-06-01
2975238   93436 2022-01-31  -11.3609  9.681319e+05   NASDAQ     2021-06-01
2975239   93436 2022-02-28   -7.0768  8.996190e+05   NASDAQ     2021-06-01
2975240   93436 2022-03-31 -100.0000  1.113736e+06   NASDAQ     2021-06-01

[191617 rows x 6 columns]


In [ ]:
print(ccmfund1990to2022)

         GVKEY  LPERMNO   datadate LINKTYPE LINKENDDT       seq       ceq  \
0         1004    54594 1991-05-31       LU       NaT   193.778   193.778   
1         1004    54594 1992-05-31       LU       NaT   196.737   196.737   
2         1004    54594 1993-05-31       LU       NaT   189.216   189.216   
3         1004    54594 1994-05-31       LU       NaT   189.488   189.488   
4         1004    54594 1995-05-31       LU       NaT   197.119   197.119   
...        ...      ...        ...      ...       ...       ...       ...   
209121  345980    20333 2020-12-31       LC       NaT  1027.000  1027.000   
209122  345980    20333 2021-12-31       LC       NaT   818.000   818.000   
209123  347007    15533 2021-12-31       LC       NaT  -242.173  -242.173   
209124  347085    21069 2022-02-28       LC       NaT   139.431   139.431   
209125  349530    17324 2022-02-28       LC       NaT    64.231    64.231   

              at        lt  txditc  ...  itcb  pstkrv  pstkl  pstk  indfmt 

In [ ]:
print(ccmfund2018to2022.columns)

Index(['gvkey', 'permno', 'datadate', 'at', 'book_value', 'OP', 'inv',
       'reference_date'],
      dtype='object')


In [ ]:

ccmfund2018to2022.rename(columns={'LPERMNO':'permno', 'ret_adj':'ret'}, inplace=True)
print(ccmfund2018to2022)

        gvkey  permno   datadate         at  book_value            OP  \
0       12096   10028 2017-12-31     13.315       7.756      9.930800   
1       12096   10028 2018-12-31     13.018       8.413      8.638047   
2       12096   10028 2019-12-31     27.003      11.194     14.732460   
3       12096   10028 2020-12-31     40.579      17.578     21.564201   
4       12096   10028 2021-12-31     59.267      27.627     29.764186   
...       ...     ...        ...        ...         ...           ...   
18088  184996   93436 2017-12-31  28655.372    4237.312   3261.424138   
18089  184996   93436 2018-12-31  29739.614    4923.243   5210.963636   
18090  184996   93436 2019-12-31  34309.000    6618.000   5459.397250   
18091  184996   93436 2020-12-31  52148.000   22376.000   8644.789471   
18092  184996   93436 2021-12-31  62131.000   30213.000  16192.764484   

            inv reference_date  
0                   2018-06-01  
1      0.977694     2019-06-01  
2      2.074282     2020

In [ ]:
#checkpoint
j= stonks2018To2022
k= ccmfund2018to2022

In [ ]:
j = j.reset_index()
j.drop('index', inplace=True, axis=1)

print(j)

        permno       date       ret    market_cap exchange reference_date
0        10026 2018-01-31   -8.8191  2.585782e+03   NASDAQ     2017-06-01
1        10026 2018-02-28   -2.9688  2.509016e+03   NASDAQ     2017-06-01
2        10026 2018-03-29    1.9951  2.553262e+03   NASDAQ     2017-06-01
3        10026 2018-04-30    0.6224  2.569842e+03   NASDAQ     2017-06-01
4        10026 2018-05-31    3.0638  2.648577e+03   NASDAQ     2017-06-01
...        ...        ...       ...           ...      ...            ...
191612   93436 2021-11-30    2.7612  1.149642e+06   NASDAQ     2021-06-01
191613   93436 2021-12-31   -7.6855  1.092218e+06   NASDAQ     2021-06-01
191614   93436 2022-01-31  -11.3609  9.681319e+05   NASDAQ     2021-06-01
191615   93436 2022-02-28   -7.0768  8.996190e+05   NASDAQ     2021-06-01
191616   93436 2022-03-31 -100.0000  1.113736e+06   NASDAQ     2021-06-01

[191617 rows x 6 columns]


In [ ]:

k= k.sort_values(by= ['permno', "reference_date"], ascending= True)
j= j.sort_values(by= ['permno', "reference_date"], ascending= True)

print(k)
print(j)

        gvkey  permno   datadate         at  book_value            OP  \
0       12096   10028 2017-12-31     13.315       7.756      9.930800   
1       12096   10028 2018-12-31     13.018       8.413      8.638047   
2       12096   10028 2019-12-31     27.003      11.194     14.732460   
3       12096   10028 2020-12-31     40.579      17.578     21.564201   
4       12096   10028 2021-12-31     59.267      27.627     29.764186   
...       ...     ...        ...        ...         ...           ...   
18088  184996   93436 2017-12-31  28655.372    4237.312   3261.424138   
18089  184996   93436 2018-12-31  29739.614    4923.243   5210.963636   
18090  184996   93436 2019-12-31  34309.000    6618.000   5459.397250   
18091  184996   93436 2020-12-31  52148.000   22376.000   8644.789471   
18092  184996   93436 2021-12-31  62131.000   30213.000  16192.764484   

            inv reference_date  
0                   2018-06-01  
1      0.977694     2019-06-01  
2      2.074282     2020

In [ ]:
#10:16

#merge ccm dataframe into stoncks dataframe
#left join

j= j.merge(k, how='left', on=['permno', 'reference_date'])


print(j)

        permno       date       ret    market_cap exchange reference_date  \
0        10026 2018-01-31   -8.8191  2.585782e+03   NASDAQ     2017-06-01   
1        10026 2018-02-28   -2.9688  2.509016e+03   NASDAQ     2017-06-01   
2        10026 2018-03-29    1.9951  2.553262e+03   NASDAQ     2017-06-01   
3        10026 2018-04-30    0.6224  2.569842e+03   NASDAQ     2017-06-01   
4        10026 2018-05-31    3.0638  2.648577e+03   NASDAQ     2017-06-01   
...        ...        ...       ...           ...      ...            ...   
191634   93436 2021-11-30    2.7612  1.149642e+06   NASDAQ     2021-06-01   
191635   93436 2021-12-31   -7.6855  1.092218e+06   NASDAQ     2021-06-01   
191636   93436 2022-01-31  -11.3609  9.681319e+05   NASDAQ     2021-06-01   
191637   93436 2022-02-28   -7.0768  8.996190e+05   NASDAQ     2021-06-01   
191638   93436 2022-03-31 -100.0000  1.113736e+06   NASDAQ     2021-06-01   

           gvkey   datadate       at  book_value           OP       inv  
0

In [ ]:
print(k)

        gvkey  permno   datadate         at  book_value            OP  \
0       12096   10028 2017-12-31     13.315       7.756      9.930800   
1       12096   10028 2018-12-31     13.018       8.413      8.638047   
2       12096   10028 2019-12-31     27.003      11.194     14.732460   
3       12096   10028 2020-12-31     40.579      17.578     21.564201   
4       12096   10028 2021-12-31     59.267      27.627     29.764186   
...       ...     ...        ...        ...         ...           ...   
18088  184996   93436 2017-12-31  28655.372    4237.312   3261.424138   
18089  184996   93436 2018-12-31  29739.614    4923.243   5210.963636   
18090  184996   93436 2019-12-31  34309.000    6618.000   5459.397250   
18091  184996   93436 2020-12-31  52148.000   22376.000   8644.789471   
18092  184996   93436 2021-12-31  62131.000   30213.000  16192.764484   

            inv reference_date  
0                   2018-06-01  
1      0.977694     2019-06-01  
2      2.074282     2020

In [ ]:
stonks2018To2022= j

robustness sort

In [ ]:
print(stonks2018To2022)

        permno       date       ret    market_cap exchange reference_date  \
0        10026 2018-01-31   -8.8191  2.585782e+03   NASDAQ     2017-06-01   
1        10026 2018-02-28   -2.9688  2.509016e+03   NASDAQ     2017-06-01   
2        10026 2018-03-29    1.9951  2.553262e+03   NASDAQ     2017-06-01   
3        10026 2018-04-30    0.6224  2.569842e+03   NASDAQ     2017-06-01   
4        10026 2018-05-31    3.0638  2.648577e+03   NASDAQ     2017-06-01   
...        ...        ...       ...           ...      ...            ...   
191634   93436 2021-11-30    2.7612  1.149642e+06   NASDAQ     2021-06-01   
191635   93436 2021-12-31   -7.6855  1.092218e+06   NASDAQ     2021-06-01   
191636   93436 2022-01-31  -11.3609  9.681319e+05   NASDAQ     2021-06-01   
191637   93436 2022-02-28   -7.0768  8.996190e+05   NASDAQ     2021-06-01   
191638   93436 2022-03-31 -100.0000  1.113736e+06   NASDAQ     2021-06-01   

           gvkey   datadate       at  book_value           OP       inv  
0

In [ ]:


stonks2018To2022['date']= pd.to_datetime(stonks2018To2022['date'])
robustBreakpoints= stonks2018To2022[(stonks2018To2022['date'].dt.month== 6) & (stonks2018To2022['exchange']== "NYSE")]

print(robustBreakpoints)

        permno       date      ret    market_cap exchange reference_date  \
212      10051 2019-06-28   2.6259     713.14600     NYSE     2019-06-01   
224      10051 2020-06-30  -9.8039     625.85208     NYSE     2020-06-01   
236      10051 2021-06-30  -2.1293     974.31648     NYSE     2021-06-01   
263      10104 2018-06-29  -5.6935  175409.68930     NYSE     2018-06-01   
275      10104 2019-06-28  12.5889  190041.60843     NYSE     2019-06-01   
...        ...        ...      ...           ...      ...            ...   
191466   93423 2021-06-30  -4.7326    3695.76576     NYSE     2021-06-01   
191481   93426 2018-06-29   1.5979     474.16635     NYSE     2018-06-01   
191493   93426 2019-06-28  13.4283     507.38744     NYSE     2019-06-01   
191505   93426 2020-06-30   4.0643     308.33152     NYSE     2020-06-01   
191517   93426 2021-06-30   4.4172     428.46148     NYSE     2021-06-01   

           gvkey   datadate          at  book_value            OP       inv  
212      

In [ ]:

robustBreakpoints= robustBreakpoints[['permno','reference_date','OP']]

print(robustBreakpoints)

        permno reference_date            OP
212      10051     2019-06-01           NaN
224      10051     2020-06-01           NaN
236      10051     2021-06-01           NaN
263      10104     2018-06-01  29920.709599
275      10104     2019-06-01  30946.653086
...        ...            ...           ...
191466   93423     2021-06-01    -88.722862
191481   93426     2018-06-01    109.071711
191493   93426     2019-06-01    131.989444
191505   93426     2020-06-01    126.307111
191517   93426     2021-06-01    119.511371

[5088 rows x 3 columns]


In [ ]:
#5:20 FF
#only data in june

#use double brackets for selecting columns
robustBreakpoints= robustBreakpoints[['permno','reference_date','OP']]
robustBreakpoints['reference_date']= pd.to_datetime(robustBreakpoints['reference_date'])
#what you doing here?
g= robustBreakpoints.groupby(['reference_date'])


#calculate 30th and 70th percentiles
#should i be using sizebreakpoints?
robustsort= stonks2018To2022[(stonks2018To2022['date'].dt.month== 6)]


robust_q30 = robustBreakpoints.groupby('reference_date')['OP'].quantile(0.3)
robust_q70 = robustBreakpoints.groupby('reference_date')['OP'].quantile(0.7)

# initializes an empty column with
robustsort['robust_q30']=""
robustsort['robust_q70']=""
#explain this- also can you indexes in pandas?
for i in robustsort.index:
    robustsort['robust_q30'][i]= g.get_group(g.get_group(robustsort['reference_date'][i]).reset_index()['reference_date'][0])['OP'].quantile(0.3)
    robustsort['robust_q70'][i]= g.get_group(g.get_group(robustsort['reference_date'][i]).reset_index()['reference_date'][0])['OP'].quantile(0.7)



Streaming output truncated to the last 5000 lines.
<ipython-input-62-a93db2a943bf>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  robustsort['robust_q30'][i]= g.get_group(g.get_group(robustsort['reference_date'][i]).reset_index()['reference_date'][0])['OP'].quantile(0.3)
<ipython-input-62-a93db2a943bf>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  robustsort['robust_q70'][i]= g.get_group(g.get_group(robustsort['reference_date'][i]).reset_index()['reference_date'][0])['OP'].quantile(0.7)
<ipython-input-62-a93db2a943bf>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame


In [ ]:
#can vectorize above code in chat gpt

#sort based on value, neatral, growth (H, M, L)

robustsort['robust_portfolio']=""
robustsort.sort_index(inplace=True)
for i in robustsort.index:
    if robustsort['OP'][i]>= robustsort['robust_q70'][i]:
       robustsort['robust_portfolio'][i]= "R"
    elif robustsort['OP'][i]> robustsort['robust_q30'][i]:
       robustsort["robust_portfolio"][i]= "W"
    else:
         robustsort["robust_portfolio"][i]= "N"

<ipython-input-63-cfb4f5eee8bd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  robustsort['robust_portfolio']=""
<ipython-input-63-cfb4f5eee8bd>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  robustsort["robust_portfolio"][i]= "N"
<ipython-input-63-cfb4f5eee8bd>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  robustsort["robust_portfolio"][i]= "W"
<ipython-input-63-cfb4f5e

In [ ]:
print(stonks2018To2022.columns)
print(stonks2018To2022)

Index(['permno', 'date', 'ret', 'market_cap', 'exchange', 'reference_date',
       'gvkey', 'datadate', 'at', 'book_value', 'OP', 'inv'],
      dtype='object')
        permno       date       ret    market_cap exchange reference_date  \
0        10026 2018-01-31   -8.8191  2.585782e+03   NASDAQ     2017-06-01   
1        10026 2018-02-28   -2.9688  2.509016e+03   NASDAQ     2017-06-01   
2        10026 2018-03-29    1.9951  2.553262e+03   NASDAQ     2017-06-01   
3        10026 2018-04-30    0.6224  2.569842e+03   NASDAQ     2017-06-01   
4        10026 2018-05-31    3.0638  2.648577e+03   NASDAQ     2017-06-01   
...        ...        ...       ...           ...      ...            ...   
191634   93436 2021-11-30    2.7612  1.149642e+06   NASDAQ     2021-06-01   
191635   93436 2021-12-31   -7.6855  1.092218e+06   NASDAQ     2021-06-01   
191636   93436 2022-01-31  -11.3609  9.681319e+05   NASDAQ     2021-06-01   
191637   93436 2022-02-28   -7.0768  8.996190e+05   NASDAQ     2021-06

In [ ]:
print(stonks2018To2022.columns)
print(stonks2018To2022)

Index(['permno', 'date', 'ret', 'market_cap', 'exchange', 'reference_date',
       'gvkey', 'datadate', 'at', 'book_value', 'OP', 'inv'],
      dtype='object')
        permno       date       ret    market_cap exchange reference_date  \
0        10026 2018-01-31   -8.8191  2.585782e+03   NASDAQ     2017-06-01   
1        10026 2018-02-28   -2.9688  2.509016e+03   NASDAQ     2017-06-01   
2        10026 2018-03-29    1.9951  2.553262e+03   NASDAQ     2017-06-01   
3        10026 2018-04-30    0.6224  2.569842e+03   NASDAQ     2017-06-01   
4        10026 2018-05-31    3.0638  2.648577e+03   NASDAQ     2017-06-01   
...        ...        ...       ...           ...      ...            ...   
191634   93436 2021-11-30    2.7612  1.149642e+06   NASDAQ     2021-06-01   
191635   93436 2021-12-31   -7.6855  1.092218e+06   NASDAQ     2021-06-01   
191636   93436 2022-01-31  -11.3609  9.681319e+05   NASDAQ     2021-06-01   
191637   93436 2022-02-28   -7.0768  8.996190e+05   NASDAQ     2021-06

market equity

In [ ]:

stonks2018To2022['date'] = pd.to_datetime(stonks2018To2022['date'])
stocksMarketEquity = stonks2018To2022[stonks2018To2022['date'].dt.month == 12].copy()
stocksMarketEquity['reference_date'] = stocksMarketEquity['date'].dt.year + 1
stocksMarketEquity['reference_date'] = stocksMarketEquity['reference_date'].astype(str) + "-06-01"
stocksMarketEquity['reference_date'] = pd.to_datetime(stocksMarketEquity['reference_date'])

In [ ]:
#10:31

#add market equity (for small vs big)

# stonks2018To2022['date']= pd.to_datetime(stonks2018To2022['date'])
# #only taking dates in the month of december?
# stocksMarketEquity= stonks2018To2022[stonks2018To2022['date'].dt.month ==12].copy()
# stocksMarketEquity.index = stocksMarketEquity.index.drop_duplicates()
# stocksMarketEquity['reference_date']= stonks2018To2022['date'].dt.year +1
# stocksMarketEquity['reference_date']= stonks2018To2022['reference_date'].apply(str)
# #add year to string
# stocksMarketEquity= stocksMarketEquity['reference_date']+ "-06-01"
# stocksMarketEquity['reference_date']= pd.to_datetime(stocksMarketEquity['reference_date'])


print(stocksMarketEquity)

        permno       date      ret    market_cap exchange reference_date  \
11       10026 2018-12-31  -7.5094  2.714533e+03   NASDAQ     2019-06-01   
23       10026 2019-12-31  -0.0838  3.482703e+03   NASDAQ     2020-06-01   
35       10026 2020-12-31   7.2598  2.945194e+03   NASDAQ     2021-06-01   
47       10026 2021-12-31  16.1173  3.015299e+03   NASDAQ     2022-06-01   
62       10028 2018-12-31  -4.3270  1.238235e+01     AMEX     2019-06-01   
...        ...        ...      ...           ...      ...            ...   
191584   93434 2021-12-31  -6.5069  1.061697e+02   NASDAQ     2022-06-01   
191599   93436 2018-12-31  -5.0445  5.744194e+04   NASDAQ     2019-06-01   
191611   93436 2019-12-31  26.7897  7.574366e+04   NASDAQ     2020-06-01   
191623   93436 2020-12-31  24.3252  6.773402e+05   NASDAQ     2021-06-01   
191635   93436 2021-12-31  -7.6855  1.092218e+06   NASDAQ     2022-06-01   

           gvkey   datadate         at  book_value           OP       inv  
11         

In [ ]:
#parse/ only selecting relevant variables
stocksMarketEquity= stocksMarketEquity[["permno", "reference_date", "market_cap"]]
#rename column market cap to market equity
stocksMarketEquity.rename(columns={'market_cap': 'market_equity'}, inplace=True)
print(stocksMarketEquity)

        permno reference_date  market_equity
11       10026     2019-06-01   2.714533e+03
23       10026     2020-06-01   3.482703e+03
35       10026     2021-06-01   2.945194e+03
47       10026     2022-06-01   3.015299e+03
62       10028     2019-06-01   1.238235e+01
...        ...            ...            ...
191584   93434     2022-06-01   1.061697e+02
191599   93436     2019-06-01   5.744194e+04
191611   93436     2020-06-01   7.574366e+04
191623   93436     2021-06-01   6.773402e+05
191635   93436     2022-06-01   1.092218e+06

[15261 rows x 3 columns]


In [ ]:
#drop duplicates (instead of select distinct)
stocksMarketEquity.drop_duplicates(inplace=True)
stocksMarketEquity['reference_date'] = pd.to_datetime(stocksMarketEquity['reference_date'])

print(stocksMarketEquity)

        permno reference_date  market_equity
11       10026     2019-06-01   2.714533e+03
23       10026     2020-06-01   3.482703e+03
35       10026     2021-06-01   2.945194e+03
47       10026     2022-06-01   3.015299e+03
62       10028     2019-06-01   1.238235e+01
...        ...            ...            ...
191584   93434     2022-06-01   1.061697e+02
191599   93436     2019-06-01   5.744194e+04
191611   93436     2020-06-01   7.574366e+04
191623   93436     2021-06-01   6.773402e+05
191635   93436     2022-06-01   1.092218e+06

[15259 rows x 3 columns]


In [ ]:
#left join marketequity to stonks
stonks2018To2022= stonks2018To2022.merge(stocksMarketEquity, on=['permno', 'reference_date'], how='left')

print(stonks2018To2022)

        permno       date       ret    market_cap exchange reference_date  \
0        10026 2018-01-31   -8.8191  2.585782e+03   NASDAQ     2017-06-01   
1        10026 2018-02-28   -2.9688  2.509016e+03   NASDAQ     2017-06-01   
2        10026 2018-03-29    1.9951  2.553262e+03   NASDAQ     2017-06-01   
3        10026 2018-04-30    0.6224  2.569842e+03   NASDAQ     2017-06-01   
4        10026 2018-05-31    3.0638  2.648577e+03   NASDAQ     2017-06-01   
...        ...        ...       ...           ...      ...            ...   
191634   93436 2021-11-30    2.7612  1.149642e+06   NASDAQ     2021-06-01   
191635   93436 2021-12-31   -7.6855  1.092218e+06   NASDAQ     2021-06-01   
191636   93436 2022-01-31  -11.3609  9.681319e+05   NASDAQ     2021-06-01   
191637   93436 2022-02-28   -7.0768  8.996190e+05   NASDAQ     2021-06-01   
191638   93436 2022-03-31 -100.0000  1.113736e+06   NASDAQ     2021-06-01   

           gvkey   datadate       at  book_value           OP       inv  \


In [ ]:
#clean stonks


stonks2018To2022 = stonks2018To2022.reset_index()
stonks2018To2022.drop('index', inplace=True, axis=1)

print(stonks2018To2022)

# stonks2018To2022 = stonks2018To2022.reset_index()
# stonks2018To2022 = stonks2018To2022.drop('index', axis=1)

# print(stonks2018To2022)

        permno       date       ret    market_cap exchange reference_date  \
0        10026 2018-01-31   -8.8191  2.585782e+03   NASDAQ     2017-06-01   
1        10026 2018-02-28   -2.9688  2.509016e+03   NASDAQ     2017-06-01   
2        10026 2018-03-29    1.9951  2.553262e+03   NASDAQ     2017-06-01   
3        10026 2018-04-30    0.6224  2.569842e+03   NASDAQ     2017-06-01   
4        10026 2018-05-31    3.0638  2.648577e+03   NASDAQ     2017-06-01   
...        ...        ...       ...           ...      ...            ...   
191634   93436 2021-11-30    2.7612  1.149642e+06   NASDAQ     2021-06-01   
191635   93436 2021-12-31   -7.6855  1.092218e+06   NASDAQ     2021-06-01   
191636   93436 2022-01-31  -11.3609  9.681319e+05   NASDAQ     2021-06-01   
191637   93436 2022-02-28   -7.0768  8.996190e+05   NASDAQ     2021-06-01   
191638   93436 2022-03-31 -100.0000  1.113736e+06   NASDAQ     2021-06-01   

           gvkey   datadate       at  book_value           OP       inv  \


In [ ]:
#11:03
#add book to market and market cap columns (for growth neutral value)


stonks2018To2022['bm_ratio']= stonks2018To2022['book_value']/ stonks2018To2022['market_equity']

#add market cap of june of year y of reference data
stonks2018To2022['date']= pd.to_datetime(stonks2018To2022['date'])
#only taking dates in the month of december?
stocksWeight= stonks2018To2022[stonks2018To2022['date'].dt.month ==6]

##parse/ only selecting relevant variables
#stocksMarketEquity= stocksMarketEquity[["PERMO", "reference_date", "market_cap"]]
##rename column market cap to market equity
#stocksMarketEquity.rename(columns={'market_cap': 'market_equity'}, inplace=True)
#drop duplicates (instead of select distinct)
stocksWeight.drop_duplicates(inplace=True)


#left join marketequity to stonks?
stonks2018To2022.merge(stocksWeight, on=['permno', 'reference_date'], how='left')

#rename column
stonks2018To2022.rename(columns={'market_cap': 'market_cap_equity'}, inplace=True)


stonks2018To2022 = stonks2018To2022.reset_index()
stonks2018To2022.drop('index', inplace=True, axis=1)

print(stonks2018To2022)


        permno       date       ret  market_cap_equity exchange  \
0        10026 2018-01-31   -8.8191       2.585782e+03   NASDAQ   
1        10026 2018-02-28   -2.9688       2.509016e+03   NASDAQ   
2        10026 2018-03-29    1.9951       2.553262e+03   NASDAQ   
3        10026 2018-04-30    0.6224       2.569842e+03   NASDAQ   
4        10026 2018-05-31    3.0638       2.648577e+03   NASDAQ   
...        ...        ...       ...                ...      ...   
191634   93436 2021-11-30    2.7612       1.149642e+06   NASDAQ   
191635   93436 2021-12-31   -7.6855       1.092218e+06   NASDAQ   
191636   93436 2022-01-31  -11.3609       9.681319e+05   NASDAQ   
191637   93436 2022-02-28   -7.0768       8.996190e+05   NASDAQ   
191638   93436 2022-03-31 -100.0000       1.113736e+06   NASDAQ   

       reference_date     gvkey   datadate       at  book_value           OP  \
0          2017-06-01       NaN        NaT      NaN         NaN          NaN   
1          2017-06-01       NaN    

<ipython-input-72-0e43785eb19a>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocksWeight.drop_duplicates(inplace=True)


In [ ]:
#breakpoint

w= stonks2018To2022

In [ ]:
stonks2018To2022= w
print(stonks2018To2022)

        permno       date       ret  market_cap_equity exchange  \
0        10026 2018-01-31   -8.8191       2.585782e+03   NASDAQ   
1        10026 2018-02-28   -2.9688       2.509016e+03   NASDAQ   
2        10026 2018-03-29    1.9951       2.553262e+03   NASDAQ   
3        10026 2018-04-30    0.6224       2.569842e+03   NASDAQ   
4        10026 2018-05-31    3.0638       2.648577e+03   NASDAQ   
...        ...        ...       ...                ...      ...   
191634   93436 2021-11-30    2.7612       1.149642e+06   NASDAQ   
191635   93436 2021-12-31   -7.6855       1.092218e+06   NASDAQ   
191636   93436 2022-01-31  -11.3609       9.681319e+05   NASDAQ   
191637   93436 2022-02-28   -7.0768       8.996190e+05   NASDAQ   
191638   93436 2022-03-31 -100.0000       1.113736e+06   NASDAQ   

       reference_date     gvkey   datadate       at  book_value           OP  \
0          2017-06-01       NaN        NaT      NaN         NaN          NaN   
1          2017-06-01       NaN    

In [ ]:
#11:11

#quick filter of null and infiite, only keep dates after 2020 (OR 2018???)

stonks2018To2022.replace([np.inf, -np.inf], np.nan, inplace=True)
stonks2018To2022= stonks2018To2022.dropna()

stonks2018To2022['date']= pd.to_datetime(stonks2018To2022['date'])
stonks2018To2022= stonks2018To2022[stonks2018To2022['date'] >= '2020-01-01 00:00:00']


print(stonks2018To2022)

        permno       date       ret  market_cap_equity exchange  \
75       10028 2020-01-31   60.7407       5.842508e+01     AMEX   
76       10028 2020-02-28   22.5806       7.161784e+01     AMEX   
77       10028 2020-03-31   -5.2632       6.784848e+01     AMEX   
78       10028 2020-04-30   44.8413       9.827260e+01     AMEX   
79       10028 2020-05-29   -1.6438       9.665716e+01     AMEX   
...        ...        ...       ...                ...      ...   
191634   93436 2021-11-30    2.7612       1.149642e+06   NASDAQ   
191635   93436 2021-12-31   -7.6855       1.092218e+06   NASDAQ   
191636   93436 2022-01-31  -11.3609       9.681319e+05   NASDAQ   
191637   93436 2022-02-28   -7.0768       8.996190e+05   NASDAQ   
191638   93436 2022-03-31 -100.0000       1.113736e+06   NASDAQ   

       reference_date     gvkey   datadate         at  book_value  \
75         2019-06-01   12096.0 2018-12-31     13.018       8.413   
76         2019-06-01   12096.0 2018-12-31     13.018    

<ipython-input-75-ad85044a9f22>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stonks2018To2022['date']= pd.to_datetime(stonks2018To2022['date'])


Factor: smb sorting

In [ ]:
#11:29
#only data in june

stonks2018To2022['date']= pd.to_datetime(stonks2018To2022['date'])
sizeBreakpoints= stonks2018To2022[(stonks2018To2022['date'].dt.month== 6) & (stonks2018To2022['exchange']== "NYSE")]
sizeBreakpoints= sizeBreakpoints[['permno','reference_date','market_cap_equity']]
sizeBreakpoints['reference_date']= pd.to_datetime(sizeBreakpoints['reference_date'])
g= sizeBreakpoints.groupby(['reference_date'])


print(sizeBreakpoints)

        permno reference_date  market_cap_equity
287      10104     2020-06-01       169606.05414
299      10104     2021-06-01       217329.28000
440      10145     2020-06-01       101480.20232
491      10158     2020-06-01          819.78780
503      10158     2021-06-01         2086.69440
...        ...            ...                ...
191415   93422     2020-06-01          312.41478
191454   93423     2020-06-01         1626.83727
191466   93423     2021-06-01         3695.76576
191505   93426     2020-06-01          308.33152
191517   93426     2021-06-01          428.46148

[1854 rows x 3 columns]


In [ ]:
#11:51
#create size sorts and add median size column


#should i be using sizebreakpoints?
sizesorts= stonks2018To2022[(stonks2018To2022['date'].dt.month== 6)]



print(sizeBreakpoints)





# sizesorts['size_median']=""
# #median or .quantile(0.5)
# for i in sizesorts.index:
#     sizesorts['size_median'][i]= g.get_group(sizesorts['reference_date'][i]).reset_index()['market_cap'].median()



        permno reference_date  market_cap_equity
287      10104     2020-06-01       169606.05414
299      10104     2021-06-01       217329.28000
440      10145     2020-06-01       101480.20232
491      10158     2020-06-01          819.78780
503      10158     2021-06-01         2086.69440
...        ...            ...                ...
191415   93422     2020-06-01          312.41478
191454   93423     2020-06-01         1626.83727
191466   93423     2021-06-01         3695.76576
191505   93426     2020-06-01          308.33152
191517   93426     2021-06-01          428.46148

[1854 rows x 3 columns]


In [ ]:
#12:15
#sort firms based on size

# sizesorts["size_portfolio"]= ""
# for i in sizesorts.index:
#   if sizesorts["market_cap"][i] > sizesorts['size_median'][i]:
#     sizesort[i]["size_portfolio"]= "B"
#   else:
#     sizesort[i]["size_portfolio"]= "S"


In [ ]:

# Calculate median for each group
sizesorts['size_median'] = sizesorts.groupby('reference_date')['market_cap_equity'].transform('median')

# Assign size portfolio values based on condition
sizesorts['size_portfolio'] = np.where(sizesorts['market_cap_equity'] > sizesorts['size_median'], 'B', 'S')

print(sizeBreakpoints)

        permno reference_date  market_cap_equity
287      10104     2020-06-01       169606.05414
299      10104     2021-06-01       217329.28000
440      10145     2020-06-01       101480.20232
491      10158     2020-06-01          819.78780
503      10158     2021-06-01         2086.69440
...        ...            ...                ...
191415   93422     2020-06-01          312.41478
191454   93423     2020-06-01         1626.83727
191466   93423     2021-06-01         3695.76576
191505   93426     2020-06-01          308.33152
191517   93426     2021-06-01          428.46148

[1854 rows x 3 columns]


<ipython-input-79-c91e6cbe4fef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizesorts['size_median'] = sizesorts.groupby('reference_date')['market_cap_equity'].transform('median')
<ipython-input-79-c91e6cbe4fef>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizesorts['size_portfolio'] = np.where(sizesorts['market_cap_equity'] > sizesorts['size_median'], 'B', 'S')


In [ ]:
#add size sorts data into stocks

#drop duplicates (instead of select distinct)
sizesorts.drop_duplicates(inplace=True)

#left join marketequity to stonks?
stonks2018To2022= stonks2018To2022.merge(sizesorts, on=['permno', 'reference_date'], how='left')

print(stonks2018To2022)


       permno     date_x     ret_x  market_cap_equity_x exchange_x  \
0       10028 2020-01-31   60.7407         5.842508e+01       AMEX   
1       10028 2020-02-28   22.5806         7.161784e+01       AMEX   
2       10028 2020-03-31   -5.2632         6.784848e+01       AMEX   
3       10028 2020-04-30   44.8413         9.827260e+01       AMEX   
4       10028 2020-05-29   -1.6438         9.665716e+01       AMEX   
...       ...        ...       ...                  ...        ...   
63098   93436 2021-11-30    2.7612         1.149642e+06     NASDAQ   
63099   93436 2021-12-31   -7.6855         1.092218e+06     NASDAQ   
63100   93436 2022-01-31  -11.3609         9.681319e+05     NASDAQ   
63101   93436 2022-02-28   -7.0768         8.996190e+05     NASDAQ   
63102   93436 2022-03-31 -100.0000         1.113736e+06     NASDAQ   

      reference_date   gvkey_x datadate_x       at_x  book_value_x  ...  \
0         2019-06-01   12096.0 2018-12-31     13.018         8.413  ...   
1        

<ipython-input-80-7e2ff494e889>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizesorts.drop_duplicates(inplace=True)


Value factor sort

In [ ]:
#12:35
#only data in june

stonks2018To2022['date']= pd.to_datetime(stonks2018To2022['date'])
valueBreakpoints= stonks2018To2022[(stonks2018To2022['date'].dt.month== 6) & (stonks2018To2022['exchange']== "NYSE")]
valueBreakpoints= valueBreakpoints[['permno','reference_date','market_cap_equity']]
sizeBreakpoints['reference_date']= pd.to_datetime(sizeBreakpoints['reference_date'])
g= sizeBreakpoints.groupby(['reference_date'])


KeyError: ignored

In [ ]:
#12:47
#calculate 30th and 70th percentiles


#should i be using sizebreakpoints?- probs not
valuesorts= stonks2018To2022[(stonks2018To2022['date'].dt.month== 6)]

# #creation of variables?
# valuesorts['value_q30']=""
# valuesorts['value_q70']=""
# #explain this
# for i in sizesorts.index:
#     valuesorts['value_q30'][i]= g.get_group(g.get_group(valuesorts['reference_date'][i]).reset_index()['reference_date'][0])['bm_ratio'].quantile(0.3)
#     valuesorts['value_q70'][i]= g.get_group(g.get_group(valuesorts['reference_date'][i]).reset_index()['reference_date'][0])['bm_ratio'].quantile(0.7)




In [ ]:
#12:53
#sort based on value, neatral, growth (H, M, L)

# valuesorts['value_portfolio']=""
# for i in valuesorts.index:
#     if valuesorts['bm_ratio'][i]>= valuesorts['value_q70'][i]:
#        valuesorts['value_portfolio'][i]= "H"
#     elif valuesorts['bm_ratio'][i]> valuesorts['value_q30'][i]:
#        valuesorts[i]["value_portfolio"]= "L"
#     else:
#          valuesorts[i]["value_portfolio"]= "M"

In [ ]:

# Assuming you have a DataFrame valuesorts

# Group the DataFrame by 'reference_date'
grouped = valuesorts.groupby('reference_date')

# Calculate the 30th and 70th percentiles for each group
valuesorts['value_q30'] = grouped['bm_ratio'].transform(lambda x: x.quantile(0.3))
valuesorts['value_q70'] = grouped['bm_ratio'].transform(lambda x: x.quantile(0.7))


print(valuesorts.columns)
print(valuesorts)

In [ ]:

# Assuming you have a DataFrame valuesorts

# Use NumPy's vectorized conditional operation to assign values
valuesorts['value_portfolio'] = np.where(valuesorts['bm_ratio'] >= valuesorts['value_q70'], 'H',
                                         np.where(valuesorts['bm_ratio'] > valuesorts['value_q30'], 'L', 'M'))

print(valuesorts.columns)
print(valuesorts)

In [ ]:
#13:01
#add size sorts data into stocks

#drop duplicates (instead of select distinct)
valuesorts.drop_duplicates(inplace=True)

#left join marketequity to stonks?
stonks2018To2022= stonks2018To2022.merge(valuesorts, on=['permno', 'reference_date'], how='left')


In [ ]:
print(stonks2018To2022.columns)
print(stonks2018To2022)

Investment inv sort

In [ ]:
#5:42 FF
#only data in june

stonks2018To2022['date']= pd.to_datetime(stonks2018To2022['date'])
#meaning of last line- creation of variable?
invBP= stonks2018To2022[(stonks2018To2022['date'].dt.month== 6) & (stonks2018To2022['exchange']== "NYSE")& (stonks2018To2022['inv'] !="")]
invBP= invBP['permno','reference_date','inv']

invBP['inv']= pd.to_numeric(invBP['inv'])
invBP['reference_date']= pd.to_datetime(invBP['reference_date'])
#what you doing here?
g= invBP.groupby(['reference_date'])

#how to transition from invBP to sort?

#calculate 30th and 70th percentiles
#should i be using sizebreakpoints?
invsort= stonks2018To2022[(stonks2018To2022['date'].dt.month== 6)]

#creation of variables?
# Calculate quantiles for each group
grouped_quantiles = invsort.groupby('reference_date')['inv'].quantile([0.3, 0.7]).unstack()

# Rename columns for clarity
grouped_quantiles.rename(columns={0.3: 'inv_q30', 0.7: 'inv_q70'}, inplace=True)

# Merge quantiles back into the original DataFrame
invsort = invsort.merge(grouped_quantiles, left_on='reference_date', right_index=True)

# Reset index if needed
invsort.reset_index(drop=True, inplace=True)


In [ ]:
#sort based on value, neatral, growth (H, M, L)

# invsort['inv_portfolio']=""
# for i in invsort.index:
#     if invsort['inv'][i]>= invsort['inv_q70'][i]:
#        invsort['inv_portfolio'][i]= "A"
#     elif invsort['inv'][i]> invsort['inv_q30'][i]:
#        invsort[i]["inv_portfolio"]= "C"
#     else:
#          invsort[i]["inv_portfolio"]= "N"


# Create a mask for condition A
condition_a = invsort['inv'] >= invsort['inv_q70']

# Create a mask for condition C
condition_c = (invsort['inv'] > invsort['inv_q30']) & (~condition_a)

# Assign values based on conditions using NumPy's 'where' function
invsort['inv_portfolio'] = np.where(condition_a, 'A', np.where(condition_c, 'C', 'N'))

In [ ]:
#add size sorts data into stocks

#drop duplicates (instead of select distinct)
invsort.drop_duplicates(inplace=True)

#left join marketequity to stonks?
stonks2018To2022= stonks2018To2022.merge(invsort, on=['permno', 'reference_date'], how='left')


In [ ]:
print(stonks2018To2022.columns)
print(stonks2018To2022)

merdging dataframes and factors

In [ ]:
#13:25
#new groupby variable for avg weighted returns
stonks2018To2022['date']= pd.to_datetime(stonks2018To2022['date'])

#5:58 FF
#what are you doing?
retsizevalue= stonks2018To2022.groupby(['date', 'size_portfolio', 'value_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['market_cap_weight'])))
retsizerobust= stonks2018To2022.groupby(['date', 'size_portfolio', 'robust_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['market_cap_weight'])))
retsizeinv= stonks2018To2022.groupby(['date', 'size_portfolio', 'inv_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['market_cap_weight'])))
#why creating if not using?
retvaluesize= stonks2018To2022.groupby(['date', 'value_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['market_cap_weight'])))
retrobustsize= stonks2018To2022.groupby(['date', 'robust_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['market_cap_weight'])))
retinvsize= stonks2018To2022.groupby(['date', 'inv_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['market_cap_weight'])))

In [ ]:
#14:01
#create factors dataframe using only only

factors=stonks2018To2022['date']
factors.drop_duplicates(inplace=True)
factors['date']= pd.to_datetime(factors['date'])

print(factors.columns)
print(factors)

In [ ]:

# factors['smb_size']=""
# #what is index?
# for i in factors.index:
#   dateofdata= factors['date'][i]
#   factors['smb_size'][i]= np.average(retsizevalue[dateofdata]["S"])- np.average(retsizevalue[dateofdata]["B"])


# factors['smb_robust']=""
# #what is index?
# for i in factors.index:
#   dateofdata= factors['date'][i]
#   factors['smb_robust'][i]= np.average(retsizerobust[dateofdata]["S"])- np.average(retsizerobust[dateofdata]["B"])


# factors['smb_inv']=""
# #what is index?
# for i in factors.index:
#   dateofdata= factors['date'][i]
#   factors['smb_inv'][i]= np.average(retsizeinv[dateofdata]["S"])- np.average(retsizeinv[dateofdata]["B"])


# factors['smb']=""
# #what is index?
# for i in factors.index:
#   dateofdata= factors['date'][i]
#   factors['smb'][i]= np.average([factors['smb_size'][i],factors['smb_robust'][i],factors['smb_inv'][i]])


# Assuming you have DataFrames 'factors', 'retsizevalue', 'retsizerobust', 'retsizeinv'

# Calculate SMB factors using vectorized operations
factors['smb_size'] = (retsizevalue.groupby('date')['S'].mean() - retsizevalue.groupby('date')['B'].mean()).reindex(factors['date']).values
factors['smb_robust'] = (retsizerobust.groupby('date')['S'].mean() - retsizerobust.groupby('date')['B'].mean()).reindex(factors['date']).values
factors['smb_inv'] = (retsizeinv.groupby('date')['S'].mean() - retsizeinv.groupby('date')['B'].mean()).reindex(factors['date']).values

# Calculate combined SMB factor
factors['smb'] = (factors['smb_size'] + factors['smb_robust'] + factors['smb_inv']) / 3

print(factors.columns)
print(factors)

In [ ]:
###DONT RUN BLOCK- what is changing btw ff3 and 5? why is order dif


#14:09
#add in three factors smb, hml, mkt ret

# factors['smb_size']=""
# #what is index?
# for i in factors.index:
#   dateofdata= factors['date'][i]
#   factors['smb_size'][i]= np.average(valweightedret[dateofdata]["S"])- np.average(valweightedret[dateofdata]["B"])
# #what does groupby do and what does line mean
# #does order matter?
# valweightedret= stonks2018To2022.groupby(['date', 'value_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['market_cap_weight'])))


# Calculate valweightedret using vectorized groupby and apply
valweightedret = stonks2018To2022.groupby(['date', 'value_portfolio', 'size_portfolio'])['ret', 'market_cap_weight'].apply(
    lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['market_cap_weight']))
).reset_index()

# Calculate SMB factors using vectorized operations
factors['smb_size'] = (
    valweightedret[valweightedret['size_portfolio'] == 'S'].groupby('date')['ret'].mean() -
    valweightedret[valweightedret['size_portfolio'] == 'B'].groupby('date')['ret'].mean()
    ).reindex(factors['date']).values

print(factors.columns)
print(factors)

    #ORRR

# valweightedret = (
#     stonks2018To2022.groupby(['date', 'value_portfolio', 'size_portfolio'])
#     .apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['market_cap_weight'])))
#     .reset_index(name='val_weighted_ret')
# )

# # Calculate SMB factors using vectorized operations
# smb_size_df = valweightedret.pivot_table(
#     index='date', columns='size_portfolio', values='val_weighted_ret', aggfunc='mean'
# )
# factors['smb_size'] = smb_size_df['S'] - smb_size_df['B']

# # Assuming you want to calculate 'smb_robust' and 'smb_inv' in a similar way
# # ...

# # Calculate combined SMB factor
# factors['smb'] = factors[['smb_size', 'smb_robust', 'smb_inv']].mean(axis=1)


In [ ]:
# factors['hml']=""
# for i in factors.index:
#   dateofdata= factors['date'][i]
#   factors['hml'][i]= np.average(retvaluesize[dateofdata]["H"])- np.average(retvaluesize[dateofdata]["L"])

# factors['rmw']=""
# for i in factors.index:
#   dateofdata= factors['date'][i]
#   factors['rmw'][i]= np.average(retrobustsize[dateofdata]["R"])- np.average(retrobustsize[dateofdata]["W"])

# factors['cma']=""
# for i in factors.index:
#   dateofdata= factors['date'][i]
#   factors['cma'][i]= np.average(retinvsize[dateofdata]["C"])- np.average(retinvsize[dateofdata]["A"])


# factors['mkt_ret']=""
# weightedavgmonthlyret= stonks2018To2022.groupby(['date']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['market_cap_weight'])))
# for i in factors.index:
#   dateofdata= factors['date'][i]
#   factors['mkt_ret'][i]= weightedavgmonthlyret[dateofdata]




# Assuming you have DataFrames 'factors' and 'stonks2018To2022'

# Calculate HML factor using vectorized operations
hml_df = retvaluesize.pivot_table(
    index='date', columns='value_portfolio', values='H', aggfunc='mean'
)
factors['hml'] = hml_df['H'] - hml_df['L']

# Calculate RMW factor using vectorized operations
rmw_df = retrobustsize.pivot_table(
    index='date', columns='robust_portfolio', values='R', aggfunc='mean'
)
factors['rmw'] = rmw_df['R'] - rmw_df['W']

# Calculate CMA factor using vectorized operations
cma_df = retinvsize.pivot_table(
    index='date', columns='inv_portfolio', values='C', aggfunc='mean'
)
factors['cma'] = cma_df['C'] - cma_df['A']

# Calculate market return factor using vectorized operations
weightedavgmonthlyret = (
    stonks2018To2022.groupby('date')
    .apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['market_cap_weight'])))
)
factors['mkt_ret'] = weightedavgmonthlyret.reindex(factors['date']).values

print(factors.columns)
print(factors)

In [ ]:
#14:43
#clean

#Take out last month- ret= 100% due to missing dlret data??
factors['date']= pd.to_datetime(factors[['date']])
factors= factors[factors['date']< '2022-03-31']

factors.drop_duplicates(inplace=True)

#substitute for select distinct
#why not including company name?
factors= factors[["date", "mkt_ret", "smb","hml", "rmw", "cma"]]

print(factors.columns)
print(factors)

In [ ]:
#15:00
#ensure factors sorted by date
factors['date']= pd.to_datetime(factors[['date']])
factors= factors.sort_values(by=['date'], ascending= True)

print(factors.columns)
print(factors)

Clean ff data

In [ ]:
#15:12
#import fama-french data

print(FamaFrenchFiveFactorsMonthly)


In [ ]:
#15:32

#rename column Date to date, etc
FFFFmonthly=FamaFrenchFiveFactorsMonthly

FFFFmonthly.rename(columns={'Date': 'date', "Mkt_RF - RF": "mkt_ff", "SMB": "smb_ff", "HML": "hml_ff", "RMW": "rmw_ff","CMA": "cma_ff"}, inplace=True)


In [ ]:
#15:35
#convert variables

FFFFmonthly[mkt_ff]= pd.to_numeric(FFFFmonthly[mkt_ff] , errors='coerce')
FFFFmonthly[smb_ff]= pd.to_numeric(FFFFmonthly[smb_ff] , errors='coerce')
FFFFmonthly[hml_ff]= pd.to_numeric(FFFFmonthly[hml_ff] , errors='coerce')
FFFFmonthly[rmw_ff]= pd.to_numeric(FFFFmonthly[rmw_ff] , errors='coerce')
FFFFmonthly[cma_ff]= pd.to_numeric(FFFFmonthly[cma_ff] , errors='coerce')


In [ ]:
#15:42
#add month and year columns to factors and FFFFmonthly
FF=FFFFmonthly

factors['date']= pd.to_datetime(factors[['date']])
factors['month']= factors['date'].dt.month
factors['year']= factors['year'].dt.year

# FF['month']= ""
# for i in FF.index:
#   #take last two digits of date (ie take the month)
#   FF['month'][i]= int(str(FF['date'][i][-2]+ str(FF['date'][i][-1])))


# FF['year']= ""
# for i in FF.index:
#   #int(str) function takes the string and turns it into an integer
#   #take first 4 digits of date (ie take the year characters)
#   FF['year'][i]= int(str(FF['date'][i][0:4]))


# Convert 'date' column to datetime
FF['date'] = pd.to_datetime(FF['date'])

# Extract month and year
FF['month'] = FF['date'].dt.month
FF['year'] = FF['date'].dt.year

print(FF.columns)
print(FF)

In [ ]:
#15:55
#merdge FF INTO factors

#order correct?
factors= factors.merge(FF, how='left', on= ['month', 'year'])


print(factors)


FF vs factors

In [ ]:
#16:24

#various percentiles print
factors.describe(percentiles=[0.1, .25, .5, .75, .9])



In [ ]:
#16:30
#KS tests
#KS tests (one-sample) if a dataset has a normal distrobution,
 #(two-sample) if two datasets have similar distrobutions
 #results are (d,f), d being the maximum difference in values, other is p-value

ks_2samp(factors['mkt'], FF['mkt_ff'])


ks_2samp(factors['smb'], FF['smb_ff'])


ks_2samp(factors['hml'], FF['hml_ff'])



In [ ]:
#16:44
#compute correlation

factors.corr(method='pearson')





MODS to three factor

In [ ]:
#merdge crsp and compustat data

In [ ]:
#2:58
#clean CRSP/compustat merged datafram

#parse? relevant variables

In [ ]:
#3:11
#convert variables pd.to_numeric

In [ ]:
#calculate vales for each collum (make calulated collum?)

In [ ]:
#table 1
#summary stat for each

In [ ]:
#table 2
#correlatin table for all factors